これはオブジェクト指向実装のためのテストプログラム

In [1]:
"""
############################ 台風発電シミュレーション ############################

##### 説明 ###################################################################

このモジュールは台風発電船の運用をシミュレーションするためのものです。
台風予想進路を作成するクラスと台風発電船のエージェントを設定するクラスから成ります。
外部入力として最低限過去の台風の座標と対応する時刻、台風を判別できる番号のデータが必要です。
それに加えてそれぞれのクラス変数に数値を定義することで動かすことができます。
時刻ごとに台風発電船の状態量を記録することでシミュレーションを可視化するデータが得られます。
最後の状態量のみ抜き出せば、1年間の運用の結果のみを見ることができます。
台風発電船の状態量についてはclass TPGshipで説明します。

##### TODOリスト #############################################################

  ・等時間幅の過去の台風の座標とそれに対応する時刻のデータ(csv)を用意する
  ・class Forecaster では2つ、class TPGshipでは3つクラス変数を定義する
  ・拠点位置、待機位置、forecast_weightの3つのインスタンス変数の定義が今回必要
  （↑初期入力で適当な値を入れてしまったほうが良い？）

#############################################################################
"""

import polars as pl
#import pandas as pd
import numpy as np
from turtle import distance
import csv
import math
import geopy as gp
import folium as fl
import matplotlib.pyplot as plt
import japanize_matplotlib
import os
from geopy.distance import geodesic
from datetime import datetime, timedelta, timezone
from dateutil import tz
import random
from matplotlib import pyplot as pyp

クラスを定義する

In [2]:
#各国や各組織の進路予報を使用することを想定して一応残しておく
#予想したデータを受け渡す役割を持つのでForecasterとした
#現状は真のデータから期間分のデータを取り出すこと以外の能力はない
class Forecaster():

    """
    ############################## class Forecaster ##############################

    [ 説明 ]

    このクラスは台風発電船が行動を判断するのに用いる「予報データ」を作成するものです。

    クラス変数で予報の範囲と大まかな精度を指定することができます。

    ##############################################################################

    引数 :
        time_step (int) : シミュレーションにおける時間の進み幅[hours]
        current_time (int) : シミュレーション上の現在時刻(unixtime)

    属性 :
        forecast_time (int) : 予報の範囲(期間)を設定する。単位は時間[hours]。
        slope (float) : 予報の誤差距離を設定する。時間を変数とする1次関数の傾きであり遠い時間
                        ほど誤差が大きくなる。
        original_data (dataflame) : 過去の台風の座標とそれに対応する時刻を保有するデータ

    #############################################################################
    """
    
    forecast_time = 24*365
    slope = 0.1

    def cal_error_radius_km(self,time_step,advance_time_hour):

        """
        ########################## def cal_error_radius_km ##########################

        [ 説明 ]

        誤差距離[km]を計算する関数です。前提として、時間の進み幅分1つ進んだ(例：時間の進み幅が6時間

        なら6時間後)の時間の誤差距離は0kmとします。設定した傾き(slope)の1次関数で現在時刻から進めた

        時間より誤差距離を算出します。

        予報の座標を出す際に正規分布に従ってoriginal_dataの座標からランダムに点を出しますが、その正

        規分布の設定における標準偏差の算出に使われる数値となります。

        ##############################################################################

        引数 :
            advance_time_hour (int) : シミュレーション上の現在時刻に対し進めた時間[hours]

        戻り値 :
            error_radius_km (int) : その時刻の正確な座標からの平均的な座標のずれ[km]

        #############################################################################
        """

        error_radius_km = self.slope*(advance_time_hour - time_step)
        return error_radius_km

    #sd : standerd deviation 標準偏差
    def cal_forecast_point_lat_sd(self,error_radius_km,original_point):

        """
        ####################### def cal_forecast_point_lat_sd #######################

        [ 説明 ]

        関数 create_forecast で用いる緯度の標準偏差を計算するための関数です。

        誤差距離がkm単位で与えられるのに対し、正規分布で座標を求めるために緯度経度の度数が単位の数値

        が必要となります。ここでは誤差距離をkm単位から度数単位に変換することを行なっています。これが

        求める標準偏差そのものになります。適度な度数の幅を定め、その幅で緯度のみをずらしながら誤差距

        離に近くなるまでずらします。誤差距離に近くなった時の緯度とずらされる前の緯度の差がkmを度数に

        置き換えた数値になります。これを緯度の標準偏差として用います。

        わざわざこのようにして求めているのは緯度1度、経度1度の長さは緯度によって変わってしまうからで

        す。緯度は比較的差が小さいですが経度は大きいため緯度に対応し都度求める必要があります。

        ##############################################################################

        引数 :
            error_radius_km (int) : その時刻の正確な座標からの平均的な座標のずれ[km](誤差距離)
            original_point (tuple) : original_dataにおける該当時刻の台風の座標
            

        戻り値 :
            lat_sd (float) : 緯度の標準偏差として用いられる値。distanceが誤差距離にほぼ等しい時
                             のtemp_latとoriginal_point_latの差

        #############################################################################
        """

        original_point_lat = original_point[0]
        original_point_lon = original_point[1]
        distance = 0

        #緯度1度分の距離がだいたい110〜112km
        #1kmの場合の度数を(1/112)とし、誤差距離の30分の1ずつ調べることとする
        split_deg_num = (1 / 112) * (error_radius_km / 30)
        temp_lat = original_point_lat
        
        while distance < error_radius_km:
            temp_lat = temp_lat + split_deg_num
            temp_point = (temp_lat,original_point_lon)

            distance = geodesic(original_point,temp_point).km   
        
        lat_sd = temp_lat - original_point_lat

        return lat_sd

    
    def cal_forecast_point_lon_sd(self,error_radius_km,original_point):

        """
        ####################### def cal_forecast_point_lon_sd #######################

        [ 説明 ]

        関数 create_forecast で用いる経度の標準偏差を計算するための関数です。

        誤差距離がkm単位で与えられるのに対し、正規分布で座標を求めるために緯度経度の度数が単位の数値

        が必要となります。ここでは誤差距離をkm単位から度数単位に変換することを行なっています。これが

        求める標準偏差そのものになります。適度な度数の幅を定め、その幅で経度のみをずらしながら誤差距

        離に近くなるまでずらします。誤差距離に近くなった時の経度とずらされる前の経度の差がkmを度数に

        置き換えた数値になります。これを経度の標準偏差として用います。

        わざわざこのようにして求めているのは緯度1度、経度1度の長さは緯度によって変わってしまうからで

        す。緯度は比較的差が小さいですが経度は大きいため緯度に対応し都度求める必要があります。

        #############################################################################

        引数 :
            error_radius_km (int) : その時刻の正確な座標からの平均的な座標のずれ[km](誤差距離)
            original_point (tuple) : original_dataにおける該当時刻の台風の座標
            

        戻り値 :
            lon_sd (float) : 緯度の標準偏差として用いられる値。distanceが誤差距離にほぼ等しい時
                             のtemp_lonとoriginal_point_lonの差

        #############################################################################
        """

        original_point_lat = original_point[0]
        original_point_lon = original_point[1]
        distance = 0

        #経度1度分の距離がだいたい北緯0で112km、北緯70で38km
        #1kmの場合の度数を(1/112)とし、誤差距離の30分の1ずつ調べることとする
        split_deg_num = (1 / 112) * (error_radius_km / 30)
        lon = original_point_lon
        
        while distance < error_radius_km:
            lon = lon + split_deg_num
            temp_point = (original_point_lat,lon)

            distance = geodesic(original_point,temp_point).km   
        
        lon_sd = lon - original_point_lon
        
        return lon_sd
    


    def create_forecast(self , time_step , current_time):

        """
        ############################ def create_forecast ############################

        [ 説明 ]

        関数 cal_error_radius_km , cal_forecast_point_lat_sd , cal_forecast_point_lon_sd
        
        を用いて、「予報データ」を作成する関数です。各時刻の台風の座標における緯度と

        経度それぞれで、original_dataの座標を平均値と関数を用いて算出した標準偏差の正規分布からラ

        ンダムに数値を出します。これによって得られた座標を予想座標として時刻、台風の番号とセットで記

        録します。current_timeからforecast_time期間分に存在するデータに対し、この処理を行なった

        ものを「予報データ」とします。

        ##############################################################################

        引数 :
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            current_time (int) : シミュレーション上の現在時刻(unixtime)
            

        戻り値 :
            forecast_data (dataflame) : 時刻、台風の番号、予想座標を持つ予報データ

        #############################################################################
        """

        unix_forecast_time = self.forecast_time * 3600
        start_forecast_time = current_time + time_step*3600
        last_forecast_time = current_time + unix_forecast_time

        forecast_true_data = self.original_data.filter((pl.col("unixtime") >= start_forecast_time)&(pl.col("unixtime") <= last_forecast_time))

        unix_list = []
        ty_num_list = []
        lat_list = []
        lon_list = []

        forecast_lat_list = []
        forecast_lon_list = []

        #unix,ty_num,lat,lonが少なくともあれば良い
        rep_num = len(forecast_true_data)
        

        if rep_num != 0:
            for i in range(rep_num):
                unix_list.append(forecast_true_data[i,"unixtime"])
                ty_num_list.append(forecast_true_data[i,"TYPHOON NUMBER"])
                lat_list.append(forecast_true_data[i,"LAT"])
                lon_list.append(forecast_true_data[i,"LON"])
                

                true_point = (lat_list[i],lon_list[i])

                advance_time_hour = (unix_list[i] - current_time)/3600
                error_radius_km = self.cal_error_radius_km(time_step,advance_time_hour)

                lat_sd = self.cal_forecast_point_lat_sd(error_radius_km,true_point)
                lon_sd = self.cal_forecast_point_lon_sd(error_radius_km,true_point)

                forecast_lat = random.gauss(lat_list[i],lat_sd)
                forecast_lon = random.gauss(lon_list[i],lon_sd)

                forecast_lat_list.append(forecast_lat)
                forecast_lon_list.append(forecast_lon)
        
        forecast_data = pl.DataFrame({"unixtime":unix_list,"TYPHOON NUMBER":ty_num_list,"TRUE_LAT":lat_list,"TRUE_LON":lon_list,"FORE_LAT":forecast_lat_list,"FORE_LON":forecast_lon_list})
        #forecast_data.columns=["unixtime","TYPHOON NUMBER","TRUE_LAT","TRUE_LON","FORE_LAT","FORE_LON"]

        
        return forecast_data

    
    



In [3]:
#TPGship = Typhoon Power Generation ship
class TPGship():
    """
    ############################### class TPGship ###############################

    [ 説明 ]

    このクラスは台風発電船を作成するクラスです。台風発電船の行動とその行動をするための条件を定義します。

    台風発電船自体の能力や状態量もここで定義されることになります。

    ##############################################################################

    引数 :
        year (int) : シミュレーションを行う年
        time_step (int) : シミュレーションにおける時間の進み幅[hours]
        current_time (int) : シミュレーション上の現在時刻(unixtime)

    属性 :
        max_storage (float) : 台風発電船の蓄電容量の上限値
        generator_output (float) : 台風発電船の定格出力
        wind_propulsion_power (float) : 通常海域で台風発電船の風力推進機で得られる平均出力
        generating_facilities_need_max_power (float) : 停止状態の発電機を船体と最大船速で推進させるために必要な出力
        max_speed_power (float) : 付加物のない船体を最大船速で進めるのに必要な出力


        storage (float) : 台風発電船のその時刻での蓄電量
        storage_percentage (float) : 台風発電船のその時刻での蓄電量の割合
        gene_elect (float) : その時刻での発電量
        loss_elect (float) : その時刻での消費電力量
        ship_state (int) : 台風発電船の状態。通常航行、待機=0,発電状態=1,台風追従=2,台風低速追従=2.5,拠点回航=3,待機位置回航=4。
        total_gene_elect (float) : その時刻までの合計発電量
        total_loss_elect (float) : その時刻までの合計消費電力量
        total_gene_time (int) : その時刻までの合計発電時間
        total_loss_time (int) : その時刻までの合計電力消費時間

        speed_kt (float) : その時刻での台風発電船の船速(kt)
        target_name (str) : 目標地点の名前。台風の場合は番号の文字列入力。
        base_lat (float) : 拠点の緯度　※現段階では外部から入力が必要。調査で適当な値が求まったらそれを初期代入する予定。
        base_lon (float) : 拠点の経度　※現段階では外部から入力が必要。調査で適当な値が求まったらそれを初期代入する予定。
        standby_lat (float) : 待機位置の緯度　※現段階では外部から入力が必要。調査で適当な値が求まったらそれを初期代入する予定。
        standby_lon (float) : 待機位置の経度　※現段階では外部から入力が必要。調査で適当な値が求まったらそれを初期代入する予定。
        ship_lat (float) : その時刻での台風発電船の緯度
        ship_lon (float) : その時刻での台風発電船の経度
        target_lat (float) : 目標地点の緯度
        target_lon (float) : 目標地点の経度
        target_distance (float) : 台風発電船から目標地点までの距離(km)
        target_TY (int) : 追従対象の台風の番号の整数入力。ない場合は0が入る。
        go_base (int) : 1の時その時の蓄電容量によらず拠点に帰る。
        next_TY_lat (float) : time_step後の目標台風の緯度。ない場合は経度と共に0
        next_TY_lon (float) : time_step後の目標台風の経度。ない場合は緯度と共に0
        next_ship_TY_dis (float) : time_step後の目標台風と台風発電船の距離(km)。ない場合はNaN。
        brance_condition (str) : 台風発電船が行動分岐のどの分岐になったかを示す

        distance_judge_hours (int) : 追従判断基準時間。発電船にとって台風が遠いか近いかを判断する基準。　※本プログラムでは使用しない
        judge_energy_storage_per (int) : 発電船が帰港判断をする蓄電割合。
        effective_range (float) : 発電船が台風下での航行となる台風中心からの距離[km]
        sub_judge_energy_storage_per (int) : 発電船が拠点経由で目的地に向かう判断をする蓄電割合。
        judge_direction (float) : 発電船が2つの目的地の方位差から行動を判断する時の基準値[度]
        standby_via_base (int) : 待機位置へ拠点を経由して向かう場合のフラグ
        judge_time_times (float) : 台風の補足地点に発電船が最大船速で到着する時間に対し台風が到着する時間が「何倍」である時追うと判断するのかの基準値

        normal_ave_speed (float) : 平常時の平均船速(kt)
        max_speed (float) : 最大船速(kt)
        TY_tracking_speed (float) : 台風を追いかける時の船速(kt)
        speed_kt (float) : その時の船速(kt)

        forecast_data (dataflame) : 各時刻の台風の予想座標がわかるデータ。台風番号、時刻、座標を持つ　※Forecasterからもらう必要がある。
        TY_start_time_list (list) : 全ての台風の発生時刻のリスト　※現段階では外部から入力が必要。調査で適当な値が求まったらそれを初期代入する予定。
        forecast_weight (float) : 台風を評価する際の式で各項につける重みの数値。他の項は(100-forecast_weight)。　※現段階では外部から入力が必要。調査で適当な値が求まったらそれを初期代入する予定。



    #############################################################################
    """

    ####################################  パラメータ  ######################################

    max_storage = 0
    generator_output = 0
    wind_propulsion_power = 0
    generating_facilities_need_max_power = 0
    max_speed_power = 0
    

    ####################################  状態量  ######################################

    #状態量の初期値入力
    def set_initial_states(self):

        """
        ############################ def set_initial_states ############################

        [ 説明 ]

        台風発電船の各種状態量に初期値を与える関数です。

        max_storage , base_lat , base_lon , standby_lat , standby_lonの数値の定義が少なくとも先に必要です。

        ##############################################################################

        """

        #船内電気関係の状態量
        self.storage = self.max_storage * 0.1
        self.storage_percentage = (self.storage / self.max_storage) * 100
        self.gene_elect = 0
        self.loss_elect = 0
        self.ship_state = 0
        self.total_gene_elect = 0
        self.total_loss_elect = 0
        self.total_gene_time = 0
        self.total_loss_time = 0


        #発電船の行動に関する状態量(現状のクラス定義では外部入力不可(更新が内部関数のため))
        self.speed_kt = 0
        self.target_name = "base station"
        self.ship_lat = self.base_lat
        self.ship_lon = self.base_lon
        self.target_lat = self.base_lat
        self.target_lon = self.base_lon
        self.target_distance = 0
        self.target_TY = 0
        self.go_base = 0
        self.TY_and_base_action = 0
        self.next_TY_lat = 0
        self.next_TY_lon = 0
        self.next_ship_TY_dis = np.nan
        self.brance_condition = "start forecast"


        
        #発電船行動用パラメータ
        #シミュレーションに風向、風速が入ってくればその場で出せる船速に変動できるようにするので将来的に消えるパラメータ
        self.nomal_ave_speed = 8
        self.max_speed = 20

        #発電船自律判断システム設定
        self.forecast_weight = 30
        self.effective_range = 100
        self.judge_energy_storage_per = 90
        self.sub_judge_energy_storage_per = 35
        self.judge_direction = 10
        self.standby_via_base = 0
        self.judge_time_times = 1.1

        
    



    ####################################  メソッド  ######################################
    
    #船の機能としては発電量計算、消費電力量計算、予報データから台風の目標地点の決定、timestep後の時刻における追従対象台風の座標取得のみ？
    #状態量を更新するような関数はメソッドではない？

    #とりあえず状態量の計算をしている関数がわかるように　#状態量計算　をつけておく
    

    def calculate_power_consumption(self,time_step):

        """
        ############################ def calculate_power_consumption ############################

        [ 説明 ]

        time_stepごとの台風発電船の消費電力量(Wh)を計算する関数です。

        ##############################################################################

        引数 :
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        戻り値 :
            energy_loss (float) : time_stepで消費される電力量[Wh]

        #############################################################################
        """
    
        #台風追従に必要な出力
        typhoon_tracking_power = (self.max_speed_power + self.generating_facilities_need_max_power)*((self.speed_kt/self.max_speed)**3) - self.wind_propulsion_power

        if typhoon_tracking_power < 0 :
            typhoon_tracking_power = 0

        #電気から動力への変換は損失なしで行える仮定
        energy_loss = typhoon_tracking_power*time_step


        return energy_loss
    

    #状態量計算
    def get_distance(self,target_position):
        
        """
        ############################ def get_distance ############################

        [ 説明 ]

        台風発電船から目標地点への距離を計算する関数です。

        ##############################################################################

        引数 :
            target_position (taple) : 目標地点の座標(緯度,経度)
            

        戻り値 :
            distance (float) : 台風発電船から目標地点への距離(km)

        #############################################################################
        """
        
        A_position = (self.ship_lat,self.ship_lon)

        #AーB間距離
        distance = geodesic(A_position,target_position).km
    
        return distance 

    
    def get_direction(self,target_position):

        """
        ############################ def get_distance ############################

        [ 説明 ]

        台風発電船から目標地点への方位を計算する関数です。

        反時計回り(左回り)を正として角度（度数法）を返します。

        ##############################################################################

        引数 :
            target_position (taple) : 目標地点の座標(緯度,経度)
            

        戻り値 :
            direction (float) : 台風発電船から目標地点への方位(度)

        #############################################################################
        """
        #北を基準に角度を定義する
        x1 = 10 + self.ship_lat #北緯(10+船の緯度)度
        y1 = 0 + self.ship_lon #東経(0+船の経度)度

        #外積計算　正なら左回り、負なら右回り
        #船の座標 (回転中心)
        x2 = self.ship_lat
        y2 = self.ship_lon

        #目標地点の座標
        x3 = target_position[0]
        y3 = target_position[1]
        
        gaiseki = (x1-x2)*(y3-y2) - (y1-y2)*(x3-x2)
        naiseki = (x1-x2)*(x3-x2) + (y1-y2)*(y3-y2)
        size12 = np.sqrt((x1-x2)**2 + (y1-y2)**2)
        size32 = np.sqrt((x3-x2)**2 + (y3-y2)**2)

        if gaiseki == 0: #直線上
            
            if naiseki < 0:
                direction = np.pi
            else:
                direction = 0
        
        elif gaiseki < 0: #右回り
            direction = -np.arccos((naiseki)/(size12*size32))

        elif gaiseki > 0: #左回り
            direction = np.arccos((naiseki)/(size12*size32))

        else:
            print("direction error")

        direction = np.rad2deg(direction)

        return direction

    #状態量計算
    def change_kt_kmh(self):

        """
        ############################ def change_kt_kmh ############################

        [ 説明 ]

        ktをkm/hに変換する関数です

        ##############################################################################
            

        戻り値 :
            speed_kmh (float) : km/hに変換された船速

        #############################################################################
        """

        speed_kmh = self.speed_kt * 1.852
    
        return speed_kmh


    #予報データから台風の目標地点の決定
    def get_target_data(self,year,current_time,time_step):

        """
        ############################ def get_target_data ############################

        [ 説明 ]

        「予報データ」(forecast_data)から目標とする台風を決める関数です。

        予想発電時間は台風発電船が台風に追いついてから台風が消滅するまで追った場合の時間です。

        消滅時間がわからない場合は発生から5日後に台風が消滅するものとして考えます。5日以上存在する場合は予報期間の最後の時刻に消滅すると仮定します。

        台風補足時間は台風発電船が予想される台風の座標に追いつくまでにかかる時間です。

        以上二つの数値を用いて評価用の数値を以下のように計算します。

        評価数値　＝　予想発電時間＊(forecast_weight) - 台風補足時間＊(100 - forecast_weight)

        これを予報データ内の全データで計算して最も評価数値が大きかったものを選びそれを返します。

        2023/05/24追記

        補足時間について、台風発電船の最大船速で到着するのにかかる時間のX倍の時間をかけなければ台風の想定到着時間に目的地に到着できない場合、

        選択肢から除外するものとする。

        Xは判断の基準値として設定されるものとする。

        ##############################################################################

        引数 :
            current_time (int) : シミュレーション上の現在時刻[unixtime]
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        戻り値 :
            target_typhoon_data (dataflame) : 追従目標に選ばれた台風のデータ。予報データから1行分のみ取り出した状態。

        #############################################################################
        """

        #台風の平均存続時間
        #今回は5日ととりあえずしてある
        TY_mean_time_to_live = 24 * 5
        TY_mean_time_to_live_unix = TY_mean_time_to_live * 3600

        ship_speed_kmh = self.change_kt_kmh()


        #unixtimeでの時間幅
        forecast_time_unix = 3600 * self.forecast_time
        last_forecast_time = int(current_time + forecast_time_unix)
        start_forecast_time = int(current_time + 3600 * time_step)


        #該当時刻内のデータの抜き出し
        typhoon_data_forecast = self.forecast_data

        #陸地認識フェーズ　陸地内に入っているデータの消去
        typhoon_data_forecast = typhoon_data_forecast.filter((((pl.col("FORE_LAT") >= 0) & (pl.col("FORE_LAT") <= 13)) & (pl.col("FORE_LON") >= 127.5)) | # p1 ~ p2
                                                            (((pl.col("FORE_LAT") >= 13) & (pl.col("FORE_LAT") <= 15)) & (pl.col("FORE_LON") >= 125)) | # p25 ~ p255
                                                            (((pl.col("FORE_LAT") >= 15) & (pl.col("FORE_LAT") <= 24)) & (pl.col("FORE_LON") >= 123)) | # p3 ~ p4
                                                            (((pl.col("FORE_LAT") >= 24) & (pl.col("FORE_LAT") <= 26)) & (pl.col("FORE_LON") >= 126)) | # p5 ~ p55
                                                            (((pl.col("FORE_LAT") >= 26) & (pl.col("FORE_LAT") <= 28)) & (pl.col("FORE_LON") >= 130.1)) | # p555 ~ p6
                                                            (((pl.col("FORE_LAT") >= 28) & (pl.col("FORE_LAT") <= 32.2)) & (pl.col("FORE_LON") >= 132.4)) | # p7 ~ p8
                                                            (((pl.col("FORE_LAT") >= 32.2) & (pl.col("FORE_LAT") <= 34)) & (pl.col("FORE_LON") >= 137.2)) | # p9 ~ p10
                                                            (((pl.col("FORE_LAT") >= 34) & (pl.col("FORE_LAT") <= 41.2)) & (pl.col("FORE_LON") >= 143)) | # p11 ~ p12
                                                            (((pl.col("FORE_LAT") >= 41.2) & (pl.col("FORE_LAT") <= 44)) & (pl.col("FORE_LON") >= 149)) | # p13 ~ p14
                                                            (((pl.col("FORE_LAT") >= 44) & (pl.col("FORE_LAT") <= 50)) & (pl.col("FORE_LON") >= 156)) | # p15 ~ p16
                                                            ((pl.col("FORE_LAT") >= 50)) # p16 ~
                                                            )

        #台風番号順に並び替え
        typhoon_data_forecast = typhoon_data_forecast.select(pl.col("*").sort_by("TYPHOON NUMBER"))

        if len(typhoon_data_forecast) != 0:
            #予報における一番若い番号の台風の取得
            TY_start_bangou = typhoon_data_forecast[0,"TYPHOON NUMBER"]
            TY_end_bangou = typhoon_data_forecast[len(typhoon_data_forecast)-1,"TYPHOON NUMBER"]
            
            #台風発生時刻の取得番号
            occurrence_time_acquisition_num = TY_start_bangou - (year - 2000)*100
            #台風番号より台風の個数を調べる
            TY_num_forecast = typhoon_data_forecast.n_unique("TYPHOON NUMBER")


            #予報期間内の台風がどの時刻まで予報されているのかを記録するリスト
            TY_forecast_end_time = []
            #欠落した番号がいた場合のリスト
            missing_num_list = []
            #各台風番号での予測終了時刻の取得
            TY_bangou = TY_start_bangou

            for i in range(TY_num_forecast):

                #番号が後なのに先に発生しているケースがあったのでその応急処置
                #if (i == TY_num_forecast -1) and (error_num == 1):
                #print("ERROR",TY_bangou,TY_end_bangou,"unixtime",current_time,"~",last_forecast_time)
                typhoon_data_by_num = typhoon_data_forecast.filter(pl.col("TYPHOON NUMBER") == TY_bangou)
                while len(typhoon_data_by_num) == 0:
                    if len(typhoon_data_by_num) == 0:
                        missing_num_list.append(TY_bangou)
                        TY_bangou = TY_bangou + 1
                        typhoon_data_by_num = typhoon_data_forecast.filter(pl.col("TYPHOON NUMBER") == TY_bangou)
                
                typhoon_data_by_num = typhoon_data_forecast.filter(pl.col("TYPHOON NUMBER") == TY_bangou)
                typhoon_data_by_num = typhoon_data_by_num.select(pl.col("*").sort_by("unixtime",descending = True))
                TY_forecast_end_time.append(typhoon_data_by_num[0,"unixtime"])

                TY_bangou = TY_bangou + 1

        
            #現在地から予測される台風の位置までの距離
            distance_list = []
            #現在地から予測される台風の位置に到着する時刻
            ship_catch_time_list = []
            #現在時刻から目的地に台風が到着するのにかかる時間
            arrival_time_list = []
            #上記二つの時間の倍率
            time_times_list = []
            #到着時から追従した場合に予測される発電量
            projected_elect_gene_time = []
            #現在地から台風の位置に到着するのに実際必要な時刻
            true_ship_catch_time_list = []


            #台風番号順に並び替えて当該時刻に発電船が到着した場合に最後まで追従できる発電時間を項目として作る
            #last_forecast_time(予報内の最終台風存続確認時刻)と最後の時刻が等しい場合には平均の存続時間で発電量を推定する
            #今回は良い方法が思いつかなかったので全データから台風発生時刻を取得する。本来は発生時刻を記録しておきたい。

            #台風発生時刻の取得
            #台風発生時刻を入れておくリスト
            TY_occurrence_time = []
            #各台風番号で開始時刻の取得
            TY_occurrence_time = self.TY_start_time_list


        

            data_num = len(typhoon_data_forecast)

            #nd_time_list = []
            #start_time_list = []
            #shori = []
            #データごとに予測発電時間を入力する
            for i in range(data_num):
                #仮の発電開始時間
                gene_start_time = typhoon_data_forecast[i,"unixtime"]
                #考える台風番号
                TY_predict_bangou = typhoon_data_forecast[i,"TYPHOON NUMBER"]

                adjustment_num = 0
                for j in range(len(missing_num_list)):
                    if TY_predict_bangou >= missing_num_list[j]:
                        adjustment_num = adjustment_num + 1

                
                #データ参照用の番号
                data_reference_num = TY_predict_bangou - TY_start_bangou - adjustment_num

                
                


                #当該台風の予報内での終了時刻
                end_time_forecast_TY = TY_forecast_end_time[data_reference_num]
                #当該台風の発生時刻
                start_time_forecast_TY = TY_occurrence_time[TY_predict_bangou - (year - 2000) * 100 - 1]
                #start_time_list.append(start_time_forecast_TY)
                #台風最終予想時刻による場合分け。予報期間終了時刻と同じ場合はその後も台風が続くものとして、平均存続時間を用いる。
                #平均存続時間よりも長く続いている台風の場合は最終予想時刻までを発電するものと仮定する。
                if (end_time_forecast_TY == last_forecast_time) and ((end_time_forecast_TY - start_time_forecast_TY) < TY_mean_time_to_live_unix):

                    #予想される発電時間[hour]を出す
                    forecast_gene_time = (start_time_forecast_TY + TY_mean_time_to_live_unix - gene_start_time) / 3600
                    #end_time_list.append(start_time_forecast_TY + TY_mean_time_to_live_unix)
                    #shori.append(1)

                else :

                    #予想期間内で発電時間[hour]を出す
                    forecast_gene_time = (end_time_forecast_TY - gene_start_time) / 3600
                    #end_time_list.append(end_time_forecast_TY)
                    #shori.append(2)


                projected_elect_gene_time.append(forecast_gene_time)


            #データフレームに予想発電時間の項目を追加する
            #typhoon_data_forecast["処理"] = shori
            #typhoon_data_forecast["予想発電開始時間"] = start_time_list
            #typhoon_data_forecast["予想発電終了時間"] = end_time_list
            typhoon_data_forecast = typhoon_data_forecast.with_columns(pl.Series(projected_elect_gene_time).alias("FORE_GENE_TIME"))


            #距離の判別させる
            for i in range(data_num):

                typhoon_posi_future = (typhoon_data_forecast[i,"FORE_LAT"] , typhoon_data_forecast[i,"FORE_LON"])
                ship_typhoon_dis = self.get_distance(typhoon_posi_future)

                #座標間の距離から到着時刻を計算する
                if ship_speed_kmh == 0:
                    ship_speed_kmh = self.max_speed * 1.852
                ship_catch_time = math.ceil(ship_typhoon_dis/ship_speed_kmh)

                #現時刻から台風がその地点に到達するまでにかかる時間を出す
                typhoon_arrival_time = int((typhoon_data_forecast[i,"unixtime"] - current_time)/3600)
                
                #arrival_time_list.append(typhoon_arrival_time)
            
                #ship_catch_time_list.append(ship_catch_time)

                time_times_list.append(ship_catch_time/typhoon_arrival_time)

                #台風の到着予定時刻と船の到着予定時刻の内遅い方をとる
                if typhoon_arrival_time > ship_catch_time:
                    true_ship_catch_time_list.append(typhoon_arrival_time)
                else:
                    true_ship_catch_time_list.append(ship_catch_time)

                distance_list.append(ship_typhoon_dis)
            


                #print(ship_typhoon_dis)
                #print(typhoon_data_forecast.loc[i,"distance"])
        
            #台風の距離を一応書いておく
            typhoon_data_forecast = typhoon_data_forecast.with_columns(pl.Series(distance_list).alias("distance"))
            #データフレームに距離の項目を追加する
            typhoon_data_forecast = typhoon_data_forecast.with_columns(pl.Series(true_ship_catch_time_list).alias("TY_CATCH_TIME"))
            #データフレームに距離の項目を追加する
            typhoon_data_forecast = typhoon_data_forecast.with_columns(pl.Series(time_times_list).alias("JUDGE_TIME_TIMES"))

            #予想発電時間と台風補足時間の差を出す
            time_difference = []
            for i in range(len(typhoon_data_forecast)):
                time_difference.append(typhoon_data_forecast[i,"FORE_GENE_TIME"] * self.forecast_weight - typhoon_data_forecast[i,"TY_CATCH_TIME"] * (100 - self.forecast_weight))
        
            #予想発電時間と台風補足時間の差をデータに追加。名前は時間対効果
            typhoon_data_forecast = typhoon_data_forecast.with_columns(pl.Series(time_difference).alias("TIME_EFFECT"))

            #基準倍数以下の時間で到達できる台風のみをのこす。[実際の到達時間] ≦ (台風の到着時間) が実際の判定基準
            typhoon_data_forecast = typhoon_data_forecast.filter(pl.col("JUDGE_TIME_TIMES") <= self.judge_time_times)

            #データを時間対効果が大きい順に並び替える
            typhoon_data_forecast = typhoon_data_forecast.select(pl.col("*").sort_by("TIME_EFFECT",descending = True))

            if len(typhoon_data_forecast) != 0:
                #出力データフレーム
                time_effect = typhoon_data_forecast[0,"TIME_EFFECT"]

                typhoon_data_forecast = typhoon_data_forecast.filter(pl.col("TIME_EFFECT") == time_effect)

                if len(typhoon_data_forecast) > 1 :
                    #データを発電時間が長い順に並び替える
                    typhoon_data_forecast = typhoon_data_forecast.select(pl.col("*").sort_by("FORE_GENE_TIME",descending = True))

                    gene_time_max = typhoon_data_forecast[0,"FORE_GENE_TIME"]
                    typhoon_data_forecast = typhoon_data_forecast.filter(pl.col("FORE_GENE_TIME") == gene_time_max)

                    if len(typhoon_data_forecast) > 1 :
                        #データを台風補足時間が短い順に並び替える
                        typhoon_data_forecast = typhoon_data_forecast.select(pl.col("*").sort_by("TY_CATCH_TIME"))

                        gene_time_max = typhoon_data_forecast[0,"TY_CATCH_TIME"]
                        typhoon_data_forecast = typhoon_data_forecast.filter(pl.col("TY_CATCH_TIME") == gene_time_max)



        return typhoon_data_forecast
    

    #timestep後の時刻における追従対象台風の座標取得
    def get_next_time_target_TY_data(self,time_step,current_time):

        """
        ############################ def get_next_time_target_TY_data ############################

        [ 説明 ]

        get_target_dataで選ばれ、追従対象となった台風のcurrent_time + time_stepの時刻での座標を取得する関数です。

        存在しない場合は空のデータフレームが返ります。

        ##############################################################################

        引数 :
            current_time (int) : シミュレーション上の現在時刻[unixtime]
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        戻り値 :
            next_time_target_data (dataflame) : 追従目標に選ばれた台風の次の時刻でのデータ

        #############################################################################
        """

        forecast_data = self.forecast_data
        next_time = int(current_time + time_step*3600)
        target_TY = int(self.target_name)
        
        next_time_target_data = forecast_data.filter((pl.col("unixtime") == next_time) & (pl.col("TYPHOON NUMBER") == target_TY))
        

        return next_time_target_data


    #状態量計算
    #次の時刻での船の座標
    def get_next_position(self, time_step):

        """
        ############################ def get_next_position ############################

        [ 説明 ]

        台風発電船の次の時刻での座標を計算するための関数です。

        現在地から目標地点まで直線に進んだ場合にいる座標を計算して返します。

        台風発電船が次の時刻で目的地に到着できる場合は座標は目的地のものになります。

        状態量が更新されるのみなのでreturnでの戻り値はありません。

        ##############################################################################

        引数 :
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        #############################################################################
        """

        target_position = (self.target_lat,self.target_lon)

        #目的地と現在地の距離
        Goal_now_distance = self.get_distance(target_position) #[km]

        #船がtime_step時間で進める距離
        advance_distance = self.change_kt_kmh() * time_step

        #緯度の差
        g_lat = self.target_lat
        n_lat = self.ship_lat

        lat_difference = g_lat - n_lat

        #経度の差
        g_lon = self.target_lon
        n_lon = self.ship_lon

        lon_difference = g_lon - n_lon

        #進める距離と目的地までの距離の比を出す
        if Goal_now_distance != 0 :
            distance_ratio = advance_distance / Goal_now_distance
        else:
            distance_ratio = 0

        #念の為の分岐
        #距離の比が1を超える場合目的地に到着できることになるので座標を目的地へ、そうでないなら当該距離進める
        
        if distance_ratio < 1 and distance_ratio > 0 :
            
            #次の時間にいるであろう緯度
            next_lat = lat_difference * distance_ratio + n_lat

            #次の時間にいるであろう経度
            next_lon = lon_difference * distance_ratio + n_lon

        else :

            #次の時間にいるであろう緯度
            next_lat = g_lat

            #次の時間にいるであろう経度
            next_lon = g_lon

        
        next_position = (next_lat , next_lon)
        self.ship_lat = next_lat
        self.ship_lon = next_lon

    
    def return_base_action(self,time_step):

        """
        ############################ def get_next_ship_state ############################

        [ 説明 ]

        台風発電船が拠点に帰港する場合の基本的な行動をまとめた関数です。

        行っていることは、目的地の設定、行動の記録、船速の決定、到着の判断です。

        ##############################################################################

        引数 :
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        #############################################################################
        """

        self.target_lat = self.base_lat
        self.target_lon = self.base_lon

        self.go_base = 1
        self.brance_condition = "battery capacity exceeded specified ratio"

        #帰港での船速の入力
        self.speed_kt = self.nomal_ave_speed

        self.target_name = "base station"
        
        base_ship_dis_time = self.get_distance((self.base_lat,self.base_lon))/self.change_kt_kmh()
        #timestep後に拠点に船がついている場合
        if base_ship_dis_time <= time_step:
            self.brance_condition = "arrival at base station"
            self.go_base = 0
            self.TY_and_base_action = 0

            self.speed_kt = 0

            #電気の積み下ろし
            self.storage = self.max_storage * 0.1
            
            #発電の有無の判断
            self.GS_gene_judge = 0 #0なら発電していない、1なら発電
            #電力消費の有無の判断
            self.GS_loss_judge = 0 #0なら消費していない、1なら消費
            

            #発電船状態入力
            self.ship_state = 0  #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4
        
        else:
            #発電の有無の判断
            self.GS_gene_judge = 0 #0なら発電していない、1なら発電
            #電力消費の有無の判断
            self.GS_loss_judge = 1 #0なら消費していない、1なら消費
            

            #発電船状態入力
            self.ship_state = 4  #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4
    

    def return_standby_action(self,time_step):

        """
        ############################ def get_next_ship_state ############################

        [ 説明 ]

        台風発電船が待機位置に向かう場合の基本的な行動をまとめた関数です。

        行っていることは、目的地の設定、行動の記録、船速の決定、到着の判断です。

        ##############################################################################

        引数 :
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        #############################################################################
        """

        self.brance_condition = "returning to standby position as no typhoon"

        self.target_lat = self.standby_lat
        self.target_lon = self.standby_lon

        self.target_name = "Standby position"
        

        self.speed_kt = self.nomal_ave_speed
        standby_ship_dis_time = self.get_distance((self.standby_lat,self.standby_lon))/self.change_kt_kmh()

        if standby_ship_dis_time <= time_step :
            self.brance_condition = "arrival at standby position"

            self.speed_kt = 0

            
            #発電の有無の判断
            self.GS_gene_judge = 0 #0なら発電していない、1なら発電
            #電力消費の有無の判断
            self.GS_loss_judge = 0 #0なら消費していない、1なら消費
            

            #発電船状態入力
            self.ship_state = 0  #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4

        else:
            #発電の有無の判断
            self.GS_gene_judge = 0 #0なら発電していない、1なら発電
            #電力消費の有無の判断
            self.GS_loss_judge = 1 #0なら消費していない、1なら消費
            

            #発電船状態入力
            self.ship_state = 4  #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4
        
    
    def typhoon_chase_action(self,time_step):

        """
        ############################ def typhoon_chase_action ############################

        [ 説明 ]

        台風発電船が台風を追従する場合の基本的な行動をまとめた関数です。

        行っていることは、目的地の設定、行動の記録、船速の決定、到着の判断です。

        追加で、拠点を経由するのかの判断も行います。

        ##############################################################################

        引数 :
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        #############################################################################
        """

        self.speed_kt = self.max_speed

        max_speed_kmh = self.change_kt_kmh()

        #GS_dis_judge = TY_tracking_speed_kmh*self.distance_judge_hours
        
        TY_tracking_speed = (self.target_TY_data[0,"distance"])/(self.target_TY_data[0,"TY_CATCH_TIME"]) #その場から台風へ時間ぴったりに着くように移動する場合の船速

        #算出したTY_tracking_speedが最大船速を超えないか判断。超える場合は最大船速に置き換え
        if TY_tracking_speed > max_speed_kmh:
            self.speed_kt = self.max_speed
        else:
            #km/hをktに変換
            self.speed_kt = TY_tracking_speed / 1.852
        
        #追従対象の台風までの距離
        GS_TY_dis = self.target_TY_data[0,"distance"]

        self.brance_condition = "tracking typhoon at maximum ship speed started"

        self.target_lat = self.target_TY_data[0,"FORE_LAT"]
        self.target_lon = self.target_TY_data[0,"FORE_LON"]

        if self.target_TY_data[0,"TY_CATCH_TIME"] <= time_step:
            self.brance_condition = "arrived at typhoon"
            self.speed_kt = self.max_speed

            self.GS_gene_judge = 1

            self.GS_loss_judge = 0

            #発電船状態入力
            self.ship_state = 1  #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4


        else:

            self.brance_condition = "tracking typhoon"

            self.GS_gene_judge = 0

            self.GS_loss_judge = 1

            #発電船状態入力
            self.ship_state = 2  #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4

            #座標間距離を用いた発電の有無のチェック用数値
            self.distance_check = 1 #1ならチェック必要
        

        #拠点を経由できるか、するかの判断フェーズ
        direction_to_TY = self.get_direction((self.target_lat,self.target_lon))
        direction_to_base = self.get_direction((self.base_lat,self.base_lon))
        direction_difference = np.abs(direction_to_TY - direction_to_base)
        targetTY_base_dis = geodesic((self.target_lat,self.target_lon),(self.base_lat,self.base_lon)).km
        need_distance = self.get_distance((self.base_lat,self.base_lon)) + targetTY_base_dis
        max_speed_kmh = self.max_speed * 1.852
        need_time_hours = need_distance/max_speed_kmh
        TY_catch_time = self.target_TY_data[0,"TY_CATCH_TIME"]

        TY_distance = self.get_distance((self.target_lat,self.target_lon))
        base_distance = self.get_distance((self.base_lat,self.base_lon))
        distance_dis = TY_distance - base_distance

        self.TY_and_base_action = 0

        if self.storage_percentage >= self.sub_judge_energy_storage_per:
            if need_time_hours <= TY_catch_time:
                #元の目的地に問題なくつけるのであれば即実行
                self.speed_kt = self.max_speed
                self.TY_and_base_action = 1 #台風に向かいながら拠点に帰港する行動のフラグ
                
                self.return_base_action

                self.brance_condition = "tracking typhoon via base"

            else:
                if direction_difference < self.judge_direction and distance_dis > 0:
                    #拠点の方が近くて、方位に大きな差がなければとりあえず経由する
                    self.speed_kt = self.max_speed
                    self.TY_and_base_action = 1 #台風に向かいながら拠点に帰港する行動のフラグ
            
                    self.return_base_action

                    self.brance_condition = "tracking typhoon via base"




    #状態量計算
    #行動判定も入っているので機能の要素も入っている？
    #全てのパラメータを次の時刻のものに変える処理
    def get_next_ship_state(self,year,current_time,time_step):

        """
        ############################ def get_next_ship_state ############################

        [ 説明 ]

        台風発電船というエージェントの行動規則そのものの設定であるとともに各分岐条件での状態量の更新を行う関数です。

        行動規則は次のように設定されています。

        1.その時刻での蓄電量の割合がX％以上なら拠点へ帰還

        2.台風がない場合待機位置へ帰還

        3.台風が存在し、追いついている場合発電

        4.台風が存在し、追従中でかつそれが近い場合最大船速で追従

        5.台風が存在し、追従中でかつそれが遠い場合低速で追従

        以上の順番で台風発電船が置かれている状況を判断し、対応した行動を台風発電船がとるように設定している。

        そして、各行動に対応した状態量の更新を行なっている。

        ##############################################################################

        引数 :
            year (int) : シミュレーションを行う年
            current_time (int) : シミュレーション上の現在時刻[unixtime]
            time_step (int) : シミュレーションにおける時間の進み幅[hours]
            

        #############################################################################
        """
        
        self.distance_check = 0
        
        
        #蓄電量X％以上の場合
        if self.storage_percentage >= self.judge_energy_storage_per or self.go_base == 1:
        #if self.go_base == 1:
            self.speed_kt = self.nomal_ave_speed
            
            self.return_base_action(time_step)

            if self.standby_via_base == 1:
                self.brance_condition = "return standby via base"

            ############  ここでデータ取得から判断させるよりも台風発電の選択肢に行った時にフラグを立てる方が良いかも？  ###############

            #追従対象の台風が存在するか判別
            self.target_TY_data = self.get_target_data(year,current_time,time_step)
            typhoon_num = len(self.target_TY_data)



            #############  近くに寄った場合に帰るという選択肢の追加  #####################

            #base_ship_dis = self.get_distance((self.base_lat,self.base_lon))




            if typhoon_num == 0 or self.storage_percentage >= self.judge_energy_storage_per: #台風がないまたは蓄電容量規定値超え
                
                #追従対象の台風がないことにする

                self.target_TY = 0

                self.next_TY_lat = 0
                self.next_TY_lon = 0
                self.next_ship_TY_dis = " "
            
            elif self.storage_percentage >= self.sub_judge_energy_storage_per: #少量の蓄電でも戻る場合の基準値を利用した場合

                if typhoon_num == 0:

                    #追従対象の台風がないことにする

                    self.target_TY = 0

                    self.next_TY_lat = 0
                    self.next_TY_lon = 0
                    self.next_ship_TY_dis = " "
                
                elif self.TY_and_base_action == 1:

                    #台風が来ているけど途中でよる場合の処理
                    self.brance_condition = "tracking typhoon via base"

                    #最大船速でとっとと戻る
                    self.speed_kt = self.max_speed

                    self.target_name = str(self.target_TY_data[0,"TYPHOON NUMBER"])
                    self.target_TY = self.target_TY_data[0,"TYPHOON NUMBER"]

                    comparison_lat = self.target_TY_data[0,"FORE_LAT"]
                    comparison_lon = self.target_TY_data[0,"FORE_LON"]

                    next_time_TY_data = self.get_next_time_target_TY_data(time_step,current_time)
                    
                    if len(next_time_TY_data) != 0:
                        self.next_TY_lat = next_time_TY_data[0,"FORE_LAT"]
                        self.next_TY_lon = next_time_TY_data[0,"FORE_LON"]
                        next_TY_locate = (self.next_TY_lat , self.next_TY_lon)

                        self.next_ship_TY_dis = self.get_distance(next_TY_locate)

                    else:
                        #追従対象の台風がないことにする
                        self.next_TY_lat = 0
                        self.next_TY_lon = 0
                        self.next_ship_TY_dis = " "


                    if target_TY_lat != comparison_lat or target_TY_lon != comparison_lon:
                        
                        #目標地点が変わりそうなら台風追従行動の方で再検討
                        self.typhoon_chase_action(time_step)


        #蓄電量90％未満の場合
        else:
            standby_via_base = 0
            
            self.next_TY_lat = 0
            self.next_TY_lon = 0
            self.next_ship_TY_dis = " "
            
            self.speed_kt = self.max_speed
            #追従対象の台風が存在するか判別
            self.target_TY_data = self.get_target_data(year,current_time,time_step)
            typhoon_num = len(self.target_TY_data)

            #待機位置へ帰還
            if typhoon_num == 0:
                
                if self.storage_percentage >= self.sub_judge_energy_storage_per:
                    self.return_base_action(time_step)
                    self.brance_condition = "return standby via base"
                    self.standby_via_base = 1
                    self.target_TY = 0
                else:
                    self.return_standby_action(time_step)
                    self.target_TY = 0

                



            #追従対象の台風が存在する場合
            elif typhoon_num >= 1:

                self.target_name = str(self.target_TY_data[0,"TYPHOON NUMBER"])
                self.target_TY = self.target_TY_data[0,"TYPHOON NUMBER"]
                

                next_time_TY_data = self.get_next_time_target_TY_data(time_step,current_time)
                
                if len(next_time_TY_data) != 0:
                    self.next_TY_lat = next_time_TY_data[0,"FORE_LAT"]
                    self.next_TY_lon = next_time_TY_data[0,"FORE_LON"]
                    next_TY_locate = (self.next_TY_lat , self.next_TY_lon)

                    self.next_ship_TY_dis = self.get_distance(next_TY_locate)

                self.typhoon_chase_action(time_step)

                target_TY_lat = self.target_TY_data[0,"FORE_LAT"]
                target_TY_lon = self.target_TY_data[0,"FORE_LAT"]
                
                ####

                


                    


                ########### 低速追従は考えないものとする ##############

                #elif target_TY_data[0,"TY_CATCH_TIME"] > judge_time and GS_TY_dis > GS_dis_judge:
                #    self.brance_condition = "typhoon is at a distance"

                #    self.speed_kt = self.max_speed

                #    GS_gene_judge = 0

                #    GS_loss_judge = 1

                #    self.brance_condition = "tracking typhoon at low speed from a distance"
                    #発電船状態入力
                #    self.ship_state = 2.5  #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4

                #    self.target_lat = target_TY_data[0,"FORE_LAT"]
                #    self.target_lon = target_TY_data[0,"FORE_LON"]
                    
                    

                    






        #次の時刻の発電船座標取得
        self.get_next_position(time_step)
        


        ##########現在時刻＋timestepの台風の座標を取得しておく##########
        #それを用いて台風の50km圏内に入っているかを考える分岐を作る
        if self.distance_check == 1:
            #next_time_TY_data = self.get_next_time_target_TY_data(time_step,current_time)

            self.distance_check = 0

            if len(next_time_TY_data) != 0:
                next_ship_locate = (self.ship_lat,self.ship_lon)

                self.next_TY_lat = next_time_TY_data[0,"FORE_LAT"]
                self.next_TY_lon = next_time_TY_data[0,"FORE_LON"]

                next_TY_locate = (self.next_TY_lat , self.next_TY_lon)

                self.next_ship_TY_dis = self.get_distance(next_TY_locate)

            if len(next_time_TY_data) != 0 and self.next_ship_TY_dis <= self.effective_range :
                self.brance_condition = "within 50km of a typhoon following"

                self.GS_gene_judge = 1
                self.GS_loss_judge = 0

                self.ship_state = 1 #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4
            
            else:
                #self.brance_condition = "beyond 50km of a typhoon following"

                self.GS_gene_judge = 0
                self.GS_loss_judge = 1

                self.ship_state = 2 #通常航行、待機 = 0 , 発電状態　= 1 , 台風追従　= 2 , 台風低速追従 = 2.5 , 拠点回航 = 3 , 待機位置回航 = 4


        ##########################################################


        ############
        
        #現在この関数での出力は次の時刻での　船の状態　追従目標　船速　座標　単位時間消費電力・発電量　保有電力　保有電力割合　目標地点との距離　となっている


        #その時刻〜次の時刻での消費電力計算
        self.loss_elect = self.calculate_power_consumption(time_step) * self.GS_loss_judge

        #その時刻〜次の時刻での発電量計算
        self.gene_elect = self.generator_output * time_step * self.GS_gene_judge

        self.total_gene_elect = self.total_gene_elect + self.gene_elect
        self.total_loss_elect = self.total_loss_elect + self.loss_elect

        self.total_gene_time = self.total_gene_time + time_step*self.GS_gene_judge
        self.total_loss_time = self.total_loss_time + time_step*self.GS_loss_judge

        #次の時刻での発電船保有電力
        self.storage = self.storage + self.gene_elect - self.loss_elect

        self.storage_percentage = self.storage / self.max_storage * 100
        

        #目標地点との距離
        target_position = (self.target_lat,self.target_lon)
        self.target_distance = self.get_distance(target_position)


In [4]:
def get_TY_start_time(year,TY_data):

    """
    ############################## def get_TY_start_time ##############################

    [ 説明 ]

    この関数は台風の発生時刻を取得するための関数です。

    本来は発生したごとに逐次記録すれば良いのですが、そのプログラムを作っても嵩張るだけだと思ったので、

    予報期間に関係なく発生時間は取得できるものとしてリスト化することにしました。

    add_unixtimeで処理したデータが必要です。

    ##############################################################################

    引数 :
        year (int) : シミュレーションを行う年
        TY_data (dataflame) : 過去の台風のデータ(unixtime追加後)

    戻り値 :
        TY_occurrence_time (list) : 各台風の発生時刻のリスト

    #############################################################################
    """

    TY_num = TY_data.n_unique("TYPHOON NUMBER")

    #台風発生時刻を入れておくリスト
    TY_occurrence_time = []

    #各台風番号で開始時刻の取得
    for i in range(TY_num):
        TY_bangou = (year - 2000)*100 + i + 1
        typhoon_data_by_num = TY_data.filter(pl.col("TYPHOON NUMBER") == TY_bangou)
        typhoon_data_by_num = typhoon_data_by_num.select(pl.col("*").sort_by("unixtime"))
        TY_occurrence_time.append(typhoon_data_by_num[0,"unixtime"])
    
    return TY_occurrence_time

In [5]:

#蓄電量の状態量のタイプわけ
def get_storage_state(storage_percentage):

    """
    ############################## def get_TY_start_time ##############################

    [ 説明 ]

    この関数は台風発電船の蓄電割合から対応する数値を返す関数です。
    
    この数値はシミュレーションの可視化の際に使われる数値です。

    ##############################################################################

    引数 :
        storage_percentage (float) : 台風発電船の蓄電割合

    戻り値 :
        20%以下→1 , 20%より多く80%より少ない→2 , 80%以上→3 , 100%以上→4

    #############################################################################
    """
    
    #蓄電量が20％以下
    if storage_percentage <= 20:
        
        return 1
    #蓄電量が100％以上
    elif storage_percentage >= 100:
        
        return 4
    #蓄電量が80％以上  
    elif storage_percentage >= 80:
        
        return 3
    #蓄電量が20％より多く、80％より少ない
    else:
        
        return 2

In [6]:
def cal_dwt(storage,storage_method):
    #載貨重量トンを算出する。単位はt。

    if storage_method == 1: #電気貯蔵
        #重量エネルギー密度1000Wh/kgの電池を使うこととする。
        dwt = storage/1000/1000


    elif storage_method == 2: #水素貯蔵
        #有機ハイドライドで水素を貯蔵することとする。
        dwt = storage/5000*0.0898/47.4

    else:
        print("cannot cal")
    

    return dwt
    



In [7]:
def cal_maxspeedpower(max_speed,storage,storage_method,body_num):

    dwt = cal_dwt(storage,storage_method)/body_num

    if storage_method == 1: #電気貯蔵
        #バルカー型
        k = 1.7
        power = k*(dwt**(2/3))*(max_speed**3)*body_num

    elif storage_method == 2: #水素貯蔵
        #タンカー型
        k = 2.2
        power = k*(dwt**(2/3))*(max_speed**3)*body_num
  
    else:
        print("cannot cal")
    
    return power



In [8]:
print((cal_maxspeedpower(20,75*10**9,2,2)*0.9+0.138*10**9*0.01)/10**6)

19.96480513672457


ここから動作確認用プログラム

In [8]:
year = 2020
TY_data = Forecaster()
time_step = 6
TY_data.year = year
UTC = timezone(timedelta(hours =+ 0),"UTC")
datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
current_time = int(datetime_1_1.timestamp())
typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
TY_data.original_data = typhoon_data


data = TY_data.create_forecast(time_step,current_time)
print(data.head())

Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.


shape: (5, 6)
┌────────────┬────────────────┬──────────┬──────────┬───────────┬────────────┐
│ unixtime   ┆ TYPHOON NUMBER ┆ TRUE_LAT ┆ TRUE_LON ┆ FORE_LAT  ┆ FORE_LON   │
│ ---        ┆ ---            ┆ ---      ┆ ---      ┆ ---       ┆ ---        │
│ i64        ┆ i64            ┆ f64      ┆ f64      ┆ f64       ┆ f64        │
╞════════════╪════════════════╪══════════╪══════════╪═══════════╪════════════╡
│ 1588917600 ┆ 2001           ┆ 6.8      ┆ 134.0    ┆ 3.580907  ┆ 136.366508 │
│ 1588939200 ┆ 2001           ┆ 6.4      ┆ 133.4    ┆ 8.596436  ┆ 136.516331 │
│ 1588960800 ┆ 2001           ┆ 6.3      ┆ 132.9    ┆ 3.210649  ┆ 134.297271 │
│ 1588982400 ┆ 2001           ┆ 6.5      ┆ 132.7    ┆ 4.167473  ┆ 130.01284  │
│ 1589004000 ┆ 2001           ┆ 6.7      ┆ 132.5    ┆ 11.745281 ┆ 136.37651  │
└────────────┴────────────────┴──────────┴──────────┴───────────┴────────────┘


In [14]:
year = 2018
time_step = 6
UTC = timezone(timedelta(hours =+ 0),"UTC")
datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
current_time = int(datetime_1_1.timestamp())
#終了時刻
datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
unixtime_12_31 = int(datetime_12_31.timestamp())
#unixtimeでの時間幅
time_step_unix = 3600 * time_step

#繰り返しの回数
record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
#台風データ設定
Forecaster.forecast_time = 24*5
Forecaster.slope = 0.0
TY_data = Forecaster()
TY_data.year = year
typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
TY_data.original_data = typhoon_data

#発電船パラメータ設定
max_speed_kt = 20
TPGship.max_storage = 66*(10**9) #蓄電容量[Wh]
TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]

ship1 = TPGship()
ship1.forecast_time = Forecaster.forecast_time


#拠点位置に関する設定
#発電船拠点位置
ship1.base_lat = 25
ship1.base_lon = 131


ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)
#待機位置に関する設定
ship1.standby_lat = 15
ship1.standby_lon = 130

ship1.set_initial_states()
ship1.sub_judge_energy_storage_per = 25
ship1.judge_time_times = 1.1

#外部初期値入力
storage_state_num = get_storage_state(ship1.storage_percentage)


for data_num in range(record_count):

    #予報データ取得
    ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

    #timestep後の発電船の状態を取得
    ship1.get_next_ship_state(year , current_time , time_step)

    #timestep後の時刻の取得
    current_time = current_time + time_step_unix


print((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
print(ship1.total_gene_time/24)




Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.


264.1503187884685
103.5


In [ ]:
2018
#沖ノ鳥島
270.36905773857546
105.25
#波照間島
256.23754702613456
100.0
#南鳥島
245.15184342377572
95.75
#沖大東島
265.6522619985913
104.0
#硫黄島
266.3155973902781
104.0

In [22]:
print((TPGship.max_speed_power*0.9+TPGship.generating_facilities_need_max_power)/10**6,TPGship.wind_propulsion_power/10**6)

18.170578392425906 1.8962864880473227


In [ ]:
yearlist = []
dwtlist = []
caplist = []
ratedlist = []
losslist = []
genelist = []
gdaylist = []
for i in range(6):
    year = 2017 + i
    time_step = 6
    UTC = timezone(timedelta(hours =+ 0),"UTC")
    datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
    current_time = int(datetime_1_1.timestamp())
    #終了時刻
    datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
    unixtime_12_31 = int(datetime_12_31.timestamp())
    #unixtimeでの時間幅
    time_step_unix = 3600 * time_step

    #繰り返しの回数
    record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
    #台風データ設定
    Forecaster.forecast_time = 24*5
    Forecaster.slope = 0.0
    TY_data = Forecaster()
    TY_data.year = year
    typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
    TY_data.original_data = typhoon_data

    #発電船パラメータ設定

    TPGship.max_storage = 66*(10**9) #蓄電容量[Wh]
    TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
    TPGship.generating_facilities_need_max_power = 0 #発電付加物分抵抗[W]
    TPGship.wind_propulsion_power = 0 #風力推進機による推進力[W]
    TPGship.max_speed_power = 14*(10**6) #船体を最大船速で進めるための出力[W]


    ship1 = TPGship()
    ship1.forecast_time = Forecaster.forecast_time


    #拠点位置に関する設定
    #発電船拠点位置
    ship1.base_lat = 20
    ship1.base_lon = 136


    ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)
    #待機位置に関する設定
    ship1.standby_lat = 10
    ship1.standby_lon = 135

    ship1.set_initial_states()
    ship1.sub_judge_energy_storage_per = 25
    ship1.judge_time_times = 1.1

    #外部初期値入力
    storage_state_num = get_storage_state(ship1.storage_percentage)


    for data_num in range(record_count):

        #予報データ取得
        ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

        #timestep後の発電船の状態を取得
        ship1.get_next_ship_state(year , current_time , time_step)

        #timestep後の時刻の取得
        current_time = current_time + time_step_unix


    #print(ship1.total_gene_elect - ship1.total_loss_elect)
    yearlist.append(year)
    dwtlist.append(cal_dwt(ship1.max_storage,2))
    caplist.append(ship1.max_storage/10**9)
    ratedlist.append(ship1.generator_output/0.8/10**9)
    losslist.append(ship1.max_speed_power/10**6)
    genelist.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
    gdaylist.append(ship1.total_gene_time/24)

data = pl.DataFrame({"YEAR":yearlist,"DWT[t]":dwtlist,"STORAGE[GWh]":caplist,"RATED OUTPUT[GW]":ratedlist,"TRUST POWER[MW]":losslist,"YEARLY POWER GENERATION[GWh]":genelist,"YEARLY POWER GENERATION DAY[day]":gdaylist})
data.write_csv('Type-A_sim_result_2017-2022.csv')

発電量基準の最適モデル調査

In [8]:
yearlist = []
dwtlist = []
caplist = []
ratedlist = []
standbylatlist = []
standbylonlist = []
genelist = []
gene2018list = []




#発電船パラメータ設定
max_speed_kt = 20

for st_n in range(1):
    TPGship.max_storage = (60 + 2*st_n)*(10**9) #蓄電容量[Wh]
    TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
    TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
    TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
    TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


    ship1 = TPGship()
    ship1.forecast_time = Forecaster.forecast_time


    #拠点位置に関する設定
    #発電船拠点位置
    ship1.base_lat = 24
    ship1.base_lon = 153

    for lat_n in range(9):
        #待機位置に関する設定
        ship1.standby_lat = lat_n+1
        for lon_n in range(10):
            ship1.standby_lon = 125 + lon_n



            gene = 0

            for i in range(5):
                year = 2018 + i
                time_step = 6
                UTC = timezone(timedelta(hours =+ 0),"UTC")
                datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
                current_time = int(datetime_1_1.timestamp())
                #終了時刻
                datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
                unixtime_12_31 = int(datetime_12_31.timestamp())
                #unixtimeでの時間幅
                time_step_unix = 3600 * time_step

                #繰り返しの回数
                record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
                #台風データ設定
                Forecaster.forecast_time = 24*5
                Forecaster.slope = 0.0
                TY_data = Forecaster()
                TY_data.year = year
                typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
                TY_data.original_data = typhoon_data

                

                ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

                ship1.set_initial_states()
                ship1.sub_judge_energy_storage_per = 25
                ship1.judge_time_times = 1.1

                #外部初期値入力
                storage_state_num = get_storage_state(ship1.storage_percentage)


                for data_num in range(record_count):

                    #予報データ取得
                    ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                    #timestep後の発電船の状態を取得
                    ship1.get_next_ship_state(year , current_time , time_step)

                    #timestep後の時刻の取得
                    current_time = current_time + time_step_unix

                gene = gene + ship1.total_gene_elect - ship1.total_loss_elect
                if year == 2018:
                    gene2018 = ship1.total_gene_elect - ship1.total_loss_elect

            #print(ship1.total_gene_elect - ship1.total_loss_elect)
            yearlist.append(year)
            dwtlist.append(cal_dwt(ship1.max_storage,2))
            caplist.append(ship1.max_storage/10**9)
            ratedlist.append(ship1.generator_output/0.8/10**9)
            standbylatlist.append(ship1.standby_lat)
            standbylonlist.append(ship1.standby_lon)
            genelist.append(gene/10**9)
            gene2018list.append(gene2018/10**9)

data = pl.DataFrame({"YEAR":yearlist,"DWT[t]":dwtlist,"STORAGE[GWh]":caplist,"RATED OUTPUT[GW]":ratedlist,"STANDBY LAT":standbylatlist,"STANDBY LON":standbylonlist,"YEARLY POWER GENERATION[GWh]":genelist,"2018 POWER GENERATION[GWh]":gene2018list})
data.write_csv('jasnaoe_result_2018-2022_Minamitori.csv')

蓄電容量の変更による発電量への影響調査

In [8]:

caplist = []

gene2017list = []
gene2018list = []
gene2019list = []
gene2020list = []
gene2021list = []
gene2022list = []



#発電船パラメータ設定
max_speed_kt = 20

for st_n in range(61):
    TPGship.max_storage = (10 + 5*st_n)*(10**9) #蓄電容量[Wh]
    TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
    TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
    TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
    TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


    ship1 = TPGship()
    ship1.forecast_time = Forecaster.forecast_time


    #拠点位置に関する設定
    #発電船拠点位置
    ship1.base_lat = 24
    ship1.base_lon = 153

    #待機位置に関する設定
    ship1.standby_lat = 24
    ship1.standby_lon = 153



    for i in range(6):
        year = 2017 + i
        time_step = 6
        UTC = timezone(timedelta(hours =+ 0),"UTC")
        datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
        current_time = int(datetime_1_1.timestamp())
        #終了時刻
        datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
        unixtime_12_31 = int(datetime_12_31.timestamp())
        #unixtimeでの時間幅
        time_step_unix = 3600 * time_step

        #繰り返しの回数
        record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
        #台風データ設定
        Forecaster.forecast_time = 24*5
        Forecaster.slope = 0.0
        TY_data = Forecaster()
        TY_data.year = year
        typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
        TY_data.original_data = typhoon_data

        

        ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

        ship1.set_initial_states()
        ship1.sub_judge_energy_storage_per = 40
        ship1.judge_time_times = 1.1

        #外部初期値入力
        storage_state_num = get_storage_state(ship1.storage_percentage)


        for data_num in range(record_count):

            #予報データ取得
            ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

            #timestep後の発電船の状態を取得
            ship1.get_next_ship_state(year , current_time , time_step)

            #timestep後の時刻の取得
            current_time = current_time + time_step_unix

        
        if year == 2017:
            gene2017 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2018:
            gene2018 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2019:
            gene2019 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2020:
            gene2020 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2021:
            gene2021 = ship1.total_gene_elect - ship1.total_loss_elect    
        elif year == 2022:
            gene2022 = ship1.total_gene_elect - ship1.total_loss_elect
            

    caplist.append(ship1.max_storage/10**9)
    gene2017list.append(gene2017/10**9)
    gene2018list.append(gene2018/10**9)
    gene2019list.append(gene2019/10**9)
    gene2020list.append(gene2020/10**9)
    gene2021list.append(gene2021/10**9)
    gene2022list.append(gene2022/10**9)
    

data = pl.DataFrame({"STORAGE[GWh]":caplist,"2017 POWER GENERATION[GWh]":gene2017list,"2018 POWER GENERATION[GWh]":gene2018list,"2019 POWER GENERATION[GWh]":gene2019list,"2020 POWER GENERATION[GWh]":gene2020list,"2021 POWER GENERATION[GWh]":gene2021list,"2022 POWER GENERATION[GWh]":gene2022list})
data.write_csv('jasnaoe_storage_and_base/jasnaoe_result_2017-2022_Minamitori_capa_40%_monohull.csv')

定方さん研究用

In [9]:
caplist = []

gene2017list = []
gene2018list = []
gene2019list = []
gene2020list = []
gene2021list = []
gene2022list = []



#発電船パラメータ設定
max_speed_kt = 20

for st_n in range(30):
    TPGship.max_storage = (10 + 10*st_n)*(10**9) #蓄電容量[Wh]
    TPGship.generator_output = 0.317*(10**9)*0.8 #定格出力[W]
    TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
    TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,1) #船体を最大船速で進めるための出力[W]
    TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


    ship1 = TPGship()
    ship1.forecast_time = Forecaster.forecast_time


    #拠点位置に関する設定
    #発電船拠点位置
    ship1.base_lat = 24
    ship1.base_lon = 153

    #待機位置に関する設定
    ship1.standby_lat = 24
    ship1.standby_lon = 153



    for i in range(6):
        year = 2017 + i
        time_step = 6
        UTC = timezone(timedelta(hours =+ 0),"UTC")
        datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
        current_time = int(datetime_1_1.timestamp())
        #終了時刻
        datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
        unixtime_12_31 = int(datetime_12_31.timestamp())
        #unixtimeでの時間幅
        time_step_unix = 3600 * time_step

        #繰り返しの回数
        record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
        #台風データ設定
        Forecaster.forecast_time = 24*5
        Forecaster.slope = 0.0
        TY_data = Forecaster()
        TY_data.year = year
        typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
        TY_data.original_data = typhoon_data

        

        ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

        ship1.set_initial_states()
        ship1.sub_judge_energy_storage_per = 25
        ship1.judge_time_times = 1.1

        #外部初期値入力
        storage_state_num = get_storage_state(ship1.storage_percentage)


        for data_num in range(record_count):

            #予報データ取得
            ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

            #timestep後の発電船の状態を取得
            ship1.get_next_ship_state(year , current_time , time_step)

            #timestep後の時刻の取得
            current_time = current_time + time_step_unix

        
        if year == 2017:
            gene2017 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2018:
            gene2018 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2019:
            gene2019 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2020:
            gene2020 = ship1.total_gene_elect - ship1.total_loss_elect
        elif year == 2021:
            gene2021 = ship1.total_gene_elect - ship1.total_loss_elect    
        elif year == 2022:
            gene2022 = ship1.total_gene_elect - ship1.total_loss_elect
            

    caplist.append(ship1.max_storage/10**9)
    gene2017list.append(gene2017/10**9)
    gene2018list.append(gene2018/10**9)
    gene2019list.append(gene2019/10**9)
    gene2020list.append(gene2020/10**9)
    gene2021list.append(gene2021/10**9)
    gene2022list.append(gene2022/10**9)
    

data = pl.DataFrame({"STORAGE[GWh]":caplist,"2017 POWER GENERATION[GWh]":gene2017list,"2018 POWER GENERATION[GWh]":gene2018list,"2019 POWER GENERATION[GWh]":gene2019list,"2020 POWER GENERATION[GWh]":gene2020list,"2021 POWER GENERATION[GWh]":gene2021list,"2022 POWER GENERATION[GWh]":gene2022list})
data.write_csv('Sadakata_result_2017-2022_Minamitori_capa.csv')

待機位置の変更による影響調査

In [ ]:
yearlist = []
model_list = []
base_list = []
E_list = []
W_list = []
S_list = []
N_list = []

for j in range(6):
    year = 2017 + j
    yearlist.append(year)


#発電船パラメータ設定
max_speed_kt = 20
TPGship.max_storage = 60*(10**9) #蓄電容量[Wh]
TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


ship1 = TPGship()
ship1.forecast_time = Forecaster.forecast_time

#拠点位置に関する設定
#発電船拠点位置
ship1.base_lat = 24
ship1.base_lon = 153

for case_n in range(6):
    

    if case_n == 0:
        ship1.standby_lat = 2
        ship1.standby_lon = 134
    elif case_n == 1:
        ship1.standby_lat = 24
        ship1.standby_lon = 153
    elif case_n == 2:
        ship1.standby_lat = 24
        ship1.standby_lon = 164
    elif case_n == 3:
        ship1.standby_lat = 24
        ship1.standby_lon = 143
    elif case_n == 4:
        ship1.standby_lat = 14
        ship1.standby_lon = 153
    elif case_n == 5:
        ship1.standby_lat = 34
        ship1.standby_lon = 153
    



    for i in range(6):
        year = 2017 + i
        time_step = 6
        UTC = timezone(timedelta(hours =+ 0),"UTC")
        datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
        current_time = int(datetime_1_1.timestamp())
        #終了時刻
        datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
        unixtime_12_31 = int(datetime_12_31.timestamp())
        #unixtimeでの時間幅
        time_step_unix = 3600 * time_step

        #繰り返しの回数
        record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
        #台風データ設定
        Forecaster.forecast_time = 24*5
        Forecaster.slope = 0.0
        TY_data = Forecaster()
        TY_data.year = year
        typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
        TY_data.original_data = typhoon_data

        

        ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

        ship1.set_initial_states()
        ship1.sub_judge_energy_storage_per = 25
        ship1.judge_time_times = 1.1

        #外部初期値入力
        storage_state_num = get_storage_state(ship1.storage_percentage)


        for data_num in range(record_count):

            #予報データ取得
            ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

            #timestep後の発電船の状態を取得
            ship1.get_next_ship_state(year , current_time , time_step)

            #timestep後の時刻の取得
            current_time = current_time + time_step_unix

        
        if case_n == 0:
            model_list.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
        elif case_n == 1:
            base_list.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
        elif case_n == 2:
            E_list.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
        elif case_n == 3:
            W_list.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
        elif case_n == 4:
            S_list.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
        elif case_n == 5:
            N_list.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            
    

data = pl.DataFrame({"YEAR":yearlist,"MODEL[GWh]":model_list,"BASE[GWh]":base_list,"E[GWh]":E_list,"W[GWh]":W_list,"S[GWh]":S_list,"N[GWh]":N_list})
data.write_csv('jasnaoe_result_2018-2022_Minamitori_standbyposition.csv')

拠点変更による影響調査

In [ ]:
list_2017 = []
list_2018 = []
list_2019 = []
list_2020 = []
list_2021 = []
list_2022 = []
model_list = []
base_lat_list = []
base_lon_list = []


#発電船パラメータ設定
max_speed_kt = 20
TPGship.max_storage = 70*(10**9) #蓄電容量[Wh]
TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,1) #船体を最大船速で進めるための出力[W]
TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


ship1 = TPGship()
ship1.forecast_time = Forecaster.forecast_time

#拠点位置に関する設定
#発電船拠点位置
for base_i in range(3):
    #拠点&待機位置
    ship1.base_lat = 20 + 5*base_i

    for base_j in range(7 - base_i):
        ship1.base_lon = 155 - 5*base_j

        ship1.standby_lat = ship1.base_lat
        ship1.standby_lon = ship1.base_lon

    


        for i in range(6):
            year = 2017 + i
            time_step = 6
            UTC = timezone(timedelta(hours =+ 0),"UTC")
            datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
            current_time = int(datetime_1_1.timestamp())
            #終了時刻
            datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
            unixtime_12_31 = int(datetime_12_31.timestamp())
            #unixtimeでの時間幅
            time_step_unix = 3600 * time_step

            #繰り返しの回数
            record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
            #台風データ設定
            Forecaster.forecast_time = 24*5
            Forecaster.slope = 0.0
            TY_data = Forecaster()
            TY_data.year = year
            typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
            TY_data.original_data = typhoon_data

            

            ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

            ship1.set_initial_states()
            ship1.sub_judge_energy_storage_per = 40
            ship1.judge_time_times = 1.1

            #外部初期値入力
            storage_state_num = get_storage_state(ship1.storage_percentage)


            for data_num in range(record_count):

                #予報データ取得
                ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                #timestep後の発電船の状態を取得
                ship1.get_next_ship_state(year , current_time , time_step)

                #timestep後の時刻の取得
                current_time = current_time + time_step_unix

            if year == 2017:
                list_2017.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2018:
                list_2018.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2019:
                list_2019.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2020:
                list_2020.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2021:
                list_2021.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2022:
                list_2022.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)

        base_lon_list.append(ship1.base_lon)
        base_lat_list.append(ship1.base_lat)
        model_list.append("STORAGE" + str(format(ship1.max_storage/10**9, '.1f'))+ "[GWh]" + "GENERATER SPECS" +str(format(ship1.generator_output/10**9, '.3f')) + "[GW]")
            
            
    

data = pl.DataFrame({"MODEL(ST,GE)":model_list,"BASE LON":base_lon_list,"BASE LAT":base_lat_list,"2017[GWh]":list_2017,"2018[GWh]":list_2018,"2019[GWh]":list_2019,"2020[GWh]":list_2020,"2021[GWh]":list_2021,"2022[GWh]":list_2022})
data.write_csv('jasnaoe_storage_and_base/'+ str(70) + 'GWh_result_2017-2022_baseposition_40%_monohull.csv')

拠点変更＋蓄電容量変更

In [8]:
list_2017 = []
list_2018 = []
list_2019 = []
list_2020 = []
list_2021 = []
list_2022 = []
model_list = []
storage_list = []
base_lat_list = []
base_lon_list = []

for storage_n in range(7):    
    


    storage = 240 + 10*storage_n
    #発電船パラメータ設定
    max_speed_kt = 20
    TPGship.max_storage = storage*(10**9) #蓄電容量[Wh]
    TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
    TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
    TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
    TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


    ship1 = TPGship()
    ship1.forecast_time = Forecaster.forecast_time

    #拠点位置に関する設定
    #発電船拠点位置
    for base_i in range(3):
        #拠点&待機位置
        ship1.base_lat = 20 + 5*base_i

        for base_j in range(7 - base_i):
            ship1.base_lon = 155 - 5*base_j

            ship1.standby_lat = ship1.base_lat
            ship1.standby_lon = ship1.base_lon

        


            for i in range(6):
                year = 2017 + i
                time_step = 6
                UTC = timezone(timedelta(hours =+ 0),"UTC")
                datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
                current_time = int(datetime_1_1.timestamp())
                #終了時刻
                datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
                unixtime_12_31 = int(datetime_12_31.timestamp())
                #unixtimeでの時間幅
                time_step_unix = 3600 * time_step

                #繰り返しの回数
                record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
                #台風データ設定
                Forecaster.forecast_time = 24*5
                Forecaster.slope = 0.0
                TY_data = Forecaster()
                TY_data.year = year
                typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
                TY_data.original_data = typhoon_data

                

                ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

                ship1.set_initial_states()
                ship1.sub_judge_energy_storage_per = 30
                ship1.judge_time_times = 1.1

                #外部初期値入力
                storage_state_num = get_storage_state(ship1.storage_percentage)


                for data_num in range(record_count):

                    #予報データ取得
                    ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                    #timestep後の発電船の状態を取得
                    ship1.get_next_ship_state(year , current_time , time_step)

                    #timestep後の時刻の取得
                    current_time = current_time + time_step_unix

                if year == 2017:
                    list_2017.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
                elif year == 2018:
                    list_2018.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
                elif year == 2019:
                    list_2019.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
                elif year == 2020:
                    list_2020.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
                elif year == 2021:
                    list_2021.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
                elif year == 2022:
                    list_2022.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)

            base_lon_list.append(ship1.base_lon)
            base_lat_list.append(ship1.base_lat)
            #model_list.append("STORAGE" + str(format(ship1.max_storage/10**9, '.1f'))+ "[GWh]" + "GENERATER SPECS" +str(format(ship1.generator_output/10**9, '.3f')) + "[GW]")
            storage_list.append(ship1.max_storage/10**9)
                
        

    #data = pl.DataFrame({"MODEL(ST,GE)":model_list,"BASE LON":base_lon_list,"BASE LAT":base_lat_list,"STORAGE[GWh]":storage_list,"2017[GWh]":list_2017,"2018[GWh]":list_2018,"2019[GWh]":list_2019,"2020[GWh]":list_2020,"2021[GWh]":list_2021,"2022[GWh]":list_2022})
    data = pl.DataFrame({"BASE LON":base_lon_list,"BASE LAT":base_lat_list,"STORAGE[GWh]":storage_list,"2017[GWh]":list_2017,"2018[GWh]":list_2018,"2019[GWh]":list_2019,"2020[GWh]":list_2020,"2021[GWh]":list_2021,"2022[GWh]":list_2022})
    data.write_csv('jasnaoe_storage_and_base/2017-2022_baseposition_30%_2.csv')

容量変更+寄港判断の蓄電割合変更

In [ ]:
sum_gene_list = []
storage_list = []
sub_judge_energy_storage_per_list = []



#発電船パラメータ設定
max_speed_kt = 20
for st_n in range(11):
    TPGship.max_storage = (50 + 10*st_n)*(10**9) #蓄電容量[Wh]

    for sjesp_n in range(18):
        sub_judge_energy_storage_per = 10 + 5*sjesp_n
        TPGship.generator_output = 0.138*2*(10**9)*0.8 #定格出力[W]
        TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
        TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
        TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


        ship1 = TPGship()
        ship1.forecast_time = Forecaster.forecast_time
        ship1.base_lat = 24
        ship1.base_lon = 153

        ship1.standby_lat = ship1.base_lat
        ship1.standby_lon = ship1.base_lon

        sum_gene = 0


        for i in range(6):
            year = 2017 + i
            time_step = 6
            UTC = timezone(timedelta(hours =+ 0),"UTC")
            datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
            current_time = int(datetime_1_1.timestamp())
            #終了時刻
            datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
            unixtime_12_31 = int(datetime_12_31.timestamp())
            #unixtimeでの時間幅
            time_step_unix = 3600 * time_step

            #繰り返しの回数
            record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
            #台風データ設定
            Forecaster.forecast_time = 24*5
            Forecaster.slope = 0.0
            TY_data = Forecaster()
            TY_data.year = year
            typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
            TY_data.original_data = typhoon_data

            

            ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

            ship1.set_initial_states()
            ship1.sub_judge_energy_storage_per = sub_judge_energy_storage_per
            ship1.judge_time_times = 1.1

            #外部初期値入力
            storage_state_num = get_storage_state(ship1.storage_percentage)


            for data_num in range(record_count):

                #予報データ取得
                ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                #timestep後の発電船の状態を取得
                ship1.get_next_ship_state(year , current_time , time_step)

                #timestep後の時刻の取得
                current_time = current_time + time_step_unix

            sum_gene = sum_gene + ship1.total_gene_elect - ship1.total_loss_elect
            
        sum_gene_list.append(sum_gene/10**9)
        storage_list.append(ship1.max_storage/10**9)
        sub_judge_energy_storage_per_list.append(sub_judge_energy_storage_per)
            
    

data = pl.DataFrame({"STORAGE[GWh]":storage_list,"SUB JUDGE ENERGY STORAGE PER[%]":sub_judge_energy_storage_per_list,"YEARLY POWER GENERATION[GWh]":sum_gene_list})
data.write_csv('jasnaoe_storage_and_base/2017-2022_storage_and_subjudge_2_monohull.csv')

横軸を蓄電容量にする場合

In [9]:
ave_gene_list = []
storage_list = []
#sub_judge_energy_storage_per_list = []
sjesp_10 = []
sjesp_15 = []
sjesp_20 = []
sjesp_25 = []
sjesp_30 = []
sjesp_35 = []
sjesp_40 = []
sjesp_45 = []
sjesp_50 = []
sjesp_55 = []
sjesp_60 = []
sjesp_65 = []
sjesp_70 = []
sjesp_75 = []
sjesp_80 = []
sjesp_85 = []
sjesp_90 = []
sjesp_95 = []



#発電船パラメータ設定
max_speed_kt = 20
for st_n in range(26):
    TPGship.max_storage = (50 + 10*st_n)*(10**9) #蓄電容量[Wh]
    storage_list.append(TPGship.max_storage/10**9)

    for sjesp_n in range(18):
        sub_judge_energy_storage_per = 10 + 5*sjesp_n
        TPGship.generator_output = 0.138*2*(10**9)*0.8 #定格出力[W]
        TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
        TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,1) #船体を最大船速で進めるための出力[W]
        TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


        ship1 = TPGship()
        ship1.forecast_time = Forecaster.forecast_time
        ship1.base_lat = 24
        ship1.base_lon = 153

        ship1.standby_lat = ship1.base_lat
        ship1.standby_lon = ship1.base_lon

        sum_gene = 0
        ave_gene = 0

        for i in range(6):
            year = 2017 + i
            time_step = 6
            UTC = timezone(timedelta(hours =+ 0),"UTC")
            datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
            current_time = int(datetime_1_1.timestamp())
            #終了時刻
            datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
            unixtime_12_31 = int(datetime_12_31.timestamp())
            #unixtimeでの時間幅
            time_step_unix = 3600 * time_step

            #繰り返しの回数
            record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
            #台風データ設定
            Forecaster.forecast_time = 24*5
            Forecaster.slope = 0.0
            TY_data = Forecaster()
            TY_data.year = year
            typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
            TY_data.original_data = typhoon_data

            

            ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

            ship1.set_initial_states()
            ship1.sub_judge_energy_storage_per = sub_judge_energy_storage_per
            ship1.judge_time_times = 1.1

            #外部初期値入力
            storage_state_num = get_storage_state(ship1.storage_percentage)


            for data_num in range(record_count):

                #予報データ取得
                ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                #timestep後の発電船の状態を取得
                ship1.get_next_ship_state(year , current_time , time_step)

                #timestep後の時刻の取得
                current_time = current_time + time_step_unix

            sum_gene = sum_gene + ship1.total_gene_elect - ship1.total_loss_elect
            
        ave_gene = sum_gene/10**9/6
        if sub_judge_energy_storage_per == 10:
            sjesp_10.append(ave_gene)
        elif sub_judge_energy_storage_per == 15:
            sjesp_15.append(ave_gene)
        elif sub_judge_energy_storage_per == 20:
            sjesp_20.append(ave_gene)
        elif sub_judge_energy_storage_per == 25:
            sjesp_25.append(ave_gene)
        elif sub_judge_energy_storage_per == 30:
            sjesp_30.append(ave_gene)
        elif sub_judge_energy_storage_per == 35:
            sjesp_35.append(ave_gene)
        elif sub_judge_energy_storage_per == 40:
            sjesp_40.append(ave_gene)
        elif sub_judge_energy_storage_per == 45:
            sjesp_45.append(ave_gene)
        elif sub_judge_energy_storage_per == 50:
            sjesp_50.append(ave_gene)
        elif sub_judge_energy_storage_per == 55:
            sjesp_55.append(ave_gene)
        elif sub_judge_energy_storage_per == 60:
            sjesp_60.append(ave_gene)
        elif sub_judge_energy_storage_per == 65:
            sjesp_65.append(ave_gene)
        elif sub_judge_energy_storage_per == 70:
            sjesp_70.append(ave_gene)
        elif sub_judge_energy_storage_per == 75:
            sjesp_75.append(ave_gene)
        elif sub_judge_energy_storage_per == 80:
            sjesp_80.append(ave_gene)
        elif sub_judge_energy_storage_per == 85:
            sjesp_85.append(ave_gene)
        elif sub_judge_energy_storage_per == 90:
            sjesp_90.append(ave_gene)
        elif sub_judge_energy_storage_per == 95:
            sjesp_95.append(ave_gene)
        
            
    

data = pl.DataFrame({"STORAGE[GWh]":storage_list,"10%":sjesp_10,"15%":sjesp_15,"20%":sjesp_20,"25%":sjesp_25,"30%":sjesp_30,"35%":sjesp_35,"40%":sjesp_40,"45%":sjesp_45,"50%":sjesp_50,"55%":sjesp_55,"60%":sjesp_60,"65%":sjesp_65,"70%":sjesp_70,"75%":sjesp_75,"80%":sjesp_80,"85%":sjesp_85,"90%":sjesp_90,"95%":sjesp_95})
data.write_csv('jasnaoe_storage_and_base/2017-2022_storage_and_subjudge_2_monohull_300GWh.csv')

年度ごとの出力バージョン

In [8]:
for i in range(5):
    year = 2018 + i

    ave_gene_list = []
    storage_list = []
    #sub_judge_energy_storage_per_list = []
    sjesp_10 = []
    sjesp_15 = []
    sjesp_20 = []
    sjesp_25 = []
    sjesp_30 = []
    sjesp_35 = []
    sjesp_40 = []
    sjesp_45 = []
    sjesp_50 = []
    sjesp_55 = []
    sjesp_60 = []
    sjesp_65 = []
    sjesp_70 = []
    sjesp_75 = []
    sjesp_80 = []
    sjesp_85 = []
    sjesp_90 = []
    sjesp_95 = []



    #発電船パラメータ設定
    max_speed_kt = 20
    for st_n in range(26):
        TPGship.max_storage = (10 + 10*st_n)*(10**9) #蓄電容量[Wh]
        storage_list.append(TPGship.max_storage/10**9)

        for sjesp_n in range(18):
            sub_judge_energy_storage_per = 10 + 5*sjesp_n
            TPGship.generator_output = 0.138*2*(10**9)*0.8 #定格出力[W]
            TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
            TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,1) #船体を最大船速で進めるための出力[W]
            TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


            ship1 = TPGship()
            ship1.forecast_time = Forecaster.forecast_time
            ship1.base_lat = 24
            ship1.base_lon = 153

            ship1.standby_lat = ship1.base_lat
            ship1.standby_lon = ship1.base_lon

            sum_gene = 0
            ave_gene = 0

            
            time_step = 6
            UTC = timezone(timedelta(hours =+ 0),"UTC")
            datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
            current_time = int(datetime_1_1.timestamp())
            #終了時刻
            datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
            unixtime_12_31 = int(datetime_12_31.timestamp())
            #unixtimeでの時間幅
            time_step_unix = 3600 * time_step

            #繰り返しの回数
            record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
            #台風データ設定
            Forecaster.forecast_time = 24*5
            Forecaster.slope = 0.0
            TY_data = Forecaster()
            TY_data.year = year
            typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
            TY_data.original_data = typhoon_data

            

            ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

            ship1.set_initial_states()
            ship1.sub_judge_energy_storage_per = sub_judge_energy_storage_per
            ship1.judge_time_times = 1.1

            #外部初期値入力
            storage_state_num = get_storage_state(ship1.storage_percentage)


            for data_num in range(record_count):

                #予報データ取得
                ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                #timestep後の発電船の状態を取得
                ship1.get_next_ship_state(year , current_time , time_step)

                #timestep後の時刻の取得
                current_time = current_time + time_step_unix

            sum_gene = sum_gene + ship1.total_gene_elect - ship1.total_loss_elect
                
            ave_gene = sum_gene/10**9
            if sub_judge_energy_storage_per == 10:
                sjesp_10.append(ave_gene)
            elif sub_judge_energy_storage_per == 15:
                sjesp_15.append(ave_gene)
            elif sub_judge_energy_storage_per == 20:
                sjesp_20.append(ave_gene)
            elif sub_judge_energy_storage_per == 25:
                sjesp_25.append(ave_gene)
            elif sub_judge_energy_storage_per == 30:
                sjesp_30.append(ave_gene)
            elif sub_judge_energy_storage_per == 35:
                sjesp_35.append(ave_gene)
            elif sub_judge_energy_storage_per == 40:
                sjesp_40.append(ave_gene)
            elif sub_judge_energy_storage_per == 45:
                sjesp_45.append(ave_gene)
            elif sub_judge_energy_storage_per == 50:
                sjesp_50.append(ave_gene)
            elif sub_judge_energy_storage_per == 55:
                sjesp_55.append(ave_gene)
            elif sub_judge_energy_storage_per == 60:
                sjesp_60.append(ave_gene)
            elif sub_judge_energy_storage_per == 65:
                sjesp_65.append(ave_gene)
            elif sub_judge_energy_storage_per == 70:
                sjesp_70.append(ave_gene)
            elif sub_judge_energy_storage_per == 75:
                sjesp_75.append(ave_gene)
            elif sub_judge_energy_storage_per == 80:
                sjesp_80.append(ave_gene)
            elif sub_judge_energy_storage_per == 85:
                sjesp_85.append(ave_gene)
            elif sub_judge_energy_storage_per == 90:
                sjesp_90.append(ave_gene)
            elif sub_judge_energy_storage_per == 95:
                sjesp_95.append(ave_gene)
            
                
        

    data = pl.DataFrame({"STORAGE[GWh]":storage_list,"10%":sjesp_10,"15%":sjesp_15,"20%":sjesp_20,"25%":sjesp_25,"30%":sjesp_30,"35%":sjesp_35,"40%":sjesp_40,"45%":sjesp_45,"50%":sjesp_50,"55%":sjesp_55,"60%":sjesp_60,"65%":sjesp_65,"70%":sjesp_70,"75%":sjesp_75,"80%":sjesp_80,"85%":sjesp_85,"90%":sjesp_90,"95%":sjesp_95})
    data.write_csv('jasnaoe_storage_and_base/' + str(year) + '_storage_and_subjudge_2_monohull_250GWh.csv')

一括貯蔵型、容量別

In [11]:
list_2017 = []
list_2018 = []
list_2019 = []
list_2020 = []
list_2021 = []
list_2022 = []
judge_per_list = []
storage_list = []
sum_gene_list = []
sub_judge_energy_storage_per_list = []


max_speed_kt = 20
for st_n in range(20):
    TPGship.max_storage = (50 + 10*st_n)*(10**9) #蓄電容量[Wh]

    for sjesp_n in range(17):
        sub_judge_energy_storage_per = 20 + 5*sjesp_n
        TPGship.generator_output = 0.138*(10**9)*0.8 #定格出力[W]
        TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W]
        TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
        TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


        ship1 = TPGship()
        ship1.forecast_time = Forecaster.forecast_time
        ship1.base_lat = 24
        ship1.base_lon = 153

        ship1.standby_lat = ship1.base_lat
        ship1.standby_lon = ship1.base_lon

        sum_gene = 0


        for i in range(6):
            year = 2017 + i
            time_step = 6
            UTC = timezone(timedelta(hours =+ 0),"UTC")
            datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
            current_time = int(datetime_1_1.timestamp())
            #終了時刻
            datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
            unixtime_12_31 = int(datetime_12_31.timestamp())
            #unixtimeでの時間幅
            time_step_unix = 3600 * time_step

            #繰り返しの回数
            record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
            #台風データ設定
            Forecaster.forecast_time = 24*5
            Forecaster.slope = 0.0
            TY_data = Forecaster()
            TY_data.year = year
            typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
            TY_data.original_data = typhoon_data

            

            ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

            ship1.set_initial_states()
            ship1.judge_energy_storage_per = sub_judge_energy_storage_per
            ship1.sub_judge_energy_storage_per = sub_judge_energy_storage_per
            ship1.judge_time_times = 1.1

            #外部初期値入力
            storage_state_num = get_storage_state(ship1.storage_percentage)


            for data_num in range(record_count):

                #予報データ取得
                ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                #timestep後の発電船の状態を取得
                ship1.get_next_ship_state(year , current_time , time_step)

                #timestep後の時刻の取得
                current_time = current_time + time_step_unix

            if year == 2017:
                list_2017.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2018:
                list_2018.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2019:
                list_2019.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2020:
                list_2020.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2021:
                list_2021.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)
            elif year == 2022:
                list_2022.append((ship1.total_gene_elect - ship1.total_loss_elect)/10**9)


            sum_gene = sum_gene + ship1.total_gene_elect - ship1.total_loss_elect
            
        sum_gene_list.append(sum_gene/10**9)
        storage_list.append(ship1.max_storage/10**9)
        sub_judge_energy_storage_per_list.append(sub_judge_energy_storage_per)
            
    

data = pl.DataFrame({"STORAGE[GWh]":storage_list,"JUDGE ENERGY STORAGE PER[%]":sub_judge_energy_storage_per_list,"YEARLY POWER GENERATION[GWh]":sum_gene_list,"2017[GWh]":list_2017,"2018[GWh]":list_2018,"2019[GWh]":list_2019,"2020[GWh]":list_2020,"2021[GWh]":list_2021,"2022[GWh]":list_2022})
data.write_csv('jasnaoe_storage_and_base/2017-2022_storage_and_mainjudge.csv')


年間運用データ出力

In [10]:
year = 2020
time_step = 6
UTC = timezone(timedelta(hours =+ 0),"UTC")
datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
current_time = int(datetime_1_1.timestamp())
#終了時刻
datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
unixtime_12_31 = int(datetime_12_31.timestamp())
#unixtimeでの時間幅
time_step_unix = 3600 * time_step

#繰り返しの回数
record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
#台風データ設定
Forecaster.forecast_time = 24*5
Forecaster.slope = 0.0
TY_data = Forecaster()
TY_data.year = year
typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
TY_data.original_data = typhoon_data

#発電船パラメータ設定

max_speed_kt = 20
TPGship.max_storage = 60*(10**9) #蓄電容量[Wh]
TPGship.generator_output = 0.138*(10**9) #定格出力[W]
TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W] (今回は定格出力の1％が停止状態での抵抗)
TPGship.wind_propulsion_power = TPGship.max_speed_power*0.1 #風力推進機による推進力[W]


ship1 = TPGship()
ship1.forecast_time = Forecaster.forecast_time


#拠点位置に関する設定
#発電船拠点位置
ship1.base_lat = 24
ship1.base_lon = 153


ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)
#待機位置に関する設定
ship1.standby_lat = 24
ship1.standby_lon = 153

#ship1.sub_judge_energy_storage_per = 20

ship1.set_initial_states()

#外部初期値入力
storage_state_num = get_storage_state(ship1.storage_percentage)





#####################################  出力用の設定  ############################################
#発電船の行動詳細
branch_condition_list = []

#台風の番号
target_typhoon_num = [] #そのときに追従している台風の番号（ない場合は0が入る）

#目標地点
target_name_list = []
target_lat_list = []
target_lon_list = []
target_dis_list = []

#台風座標
TY_lat_list = []
TY_lon_list = []

#発電船台風間距離
GS_TY_dis_list = []

#発電船の座標
GS_lat_list = []
GS_lon_list = []

#時刻関係
unix = [] #unixtime
date = [] #datetime

#発電船の状態
GS_state_list = [] #発電船の行動状態(描画用数値)
GS_speed_list = []

############################# 発電指数 ###############################
GS_elect_storage_percentage = [] #船内蓄電割合
GS_storage_state = []
gene_elect_time = [] #発電時間
total_gene_elect = [] #総発電量
loss_elect_time = [] #電力消費時間（航行時間）
total_loss_elect = [] #総消費電力
balance_gene_elect = [] #発電収支（船内蓄電量）
per_timestep_gene_elect = [] #時間幅あたりの発電量
per_timestep_loss_elect = [] #時間幅あたりの消費電力
year_round_balance_gene_elect  = [] #通年発電収支


#######################################  出力用リストへ入力  ###########################################

branch_condition_list.append(ship1.brance_condition)
unix.append(current_time)
date.append(datetime.fromtimestamp(unix[-1],UTC))

target_name_list.append(ship1.target_name)
target_lat_list.append(ship1.target_lat)
target_lon_list.append(ship1.target_lon)
target_dis_list.append(ship1.target_distance)

target_typhoon_num.append(ship1.target_TY)
TY_lat_list.append(ship1.next_TY_lat)
TY_lon_list.append(ship1.next_TY_lon)
GS_TY_dis_list.append(ship1.next_ship_TY_dis)

GS_lat_list.append(ship1.ship_lat)
GS_lon_list.append(ship1.ship_lon)
GS_state_list.append(ship1.ship_state)
GS_speed_list.append(ship1.speed_kt)

per_timestep_gene_elect.append(ship1.gene_elect) #時間幅あたりの発電量[Wh]
gene_elect_time.append(ship1.total_gene_time) #発電時間[hour]
total_gene_elect.append(ship1.total_gene_elect) #総発電量[Wh]

per_timestep_loss_elect.append(ship1.loss_elect) #時間幅あたりの消費電力[Wh]
loss_elect_time.append(ship1.total_loss_time) #電力消費時間（航行時間）[hour]
total_loss_elect.append(ship1.total_loss_elect) #総消費電力[Wh]

ship1.storage_percentage = (ship1.storage / ship1.max_storage) * 100
ship1.storage_state = get_storage_state(ship1.storage_percentage)
GS_elect_storage_percentage.append(ship1.storage_percentage) #船内蓄電割合[%]
GS_storage_state.append(ship1.storage_state)

balance_gene_elect.append(ship1.storage) #発電収支（船内蓄電量）[Wh]

year_round_balance_gene_elect.append(ship1.total_gene_elect - ship1.total_loss_elect) #通年発電収支


GS_data = pl.DataFrame({"unixtime":unix,"datetime":date
                            ,"TARGET LOCATION":target_name_list,"TARGET LAT":target_lat_list,"TARGET LON":target_lon_list,"TARGET DISTANCE[km]":target_dis_list
                            ,"TARGET TYPHOON":target_typhoon_num,"TARGET TY LAT":TY_lat_list,"TARGET TY LON":TY_lon_list
                            ,"TPGSHIP LAT":GS_lat_list,"TPGSHIP LON":GS_lon_list,"TPG_TY DISTANCE[km]":GS_TY_dis_list,"BRANCH CONDITION":branch_condition_list,"TPGSHIP STATUS":GS_state_list,"SHIP SPEED[kt]":GS_speed_list
                            ,"TIMESTEP POWER GENERATION[Wh]":per_timestep_gene_elect,"TOTAL GENE TIME[h]":gene_elect_time,"TOTAL POWER GENERATION[Wh]":total_gene_elect
                            ,"TIMESTEP POWER CONSUMPTION[Wh]":per_timestep_loss_elect,"TOTAL CONS TIME[h]":loss_elect_time,"TOTAL POWER CONSUMPTION[Wh]":total_loss_elect
                            ,"ONBOARD POWER STORAGE PER[%]":GS_elect_storage_percentage,"ONBOARD POWER STORAGE STATUS":GS_storage_state,"ONBOARD ENERGY STORAGE[Wh]":balance_gene_elect,"YEARLY POWER GENERATION BALANCE":year_round_balance_gene_elect})
                             


for data_num in range(record_count):

    #予報データ取得
    ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

    #timestep後の発電船の状態を取得
    ship1.get_next_ship_state(year , current_time , time_step)

    #timestep後の時刻の取得
    current_time = current_time + time_step_unix

    #######################################  出力用リストへ入力  ###########################################

    branch_condition_list.append(ship1.brance_condition)
    unix.append(current_time)
    date.append(datetime.fromtimestamp(unix[-1],UTC))

    target_name_list.append(ship1.target_name)
    target_lat_list.append(ship1.target_lat)
    target_lon_list.append(ship1.target_lon)
    target_dis_list.append(ship1.target_distance)

    target_typhoon_num.append(ship1.target_TY)
    TY_lat_list.append(ship1.next_TY_lat)
    TY_lon_list.append(ship1.next_TY_lon)
    GS_TY_dis_list.append(ship1.next_ship_TY_dis)

    GS_lat_list.append(ship1.ship_lat)
    GS_lon_list.append(ship1.ship_lon)
    GS_state_list.append(ship1.ship_state)
    GS_speed_list.append(ship1.speed_kt)

    per_timestep_gene_elect.append(ship1.gene_elect) #時間幅あたりの発電量[Wh]
    gene_elect_time.append(ship1.total_gene_time) #発電時間[hour]
    total_gene_elect.append(ship1.total_gene_elect) #総発電量[Wh]

    per_timestep_loss_elect.append(ship1.loss_elect) #時間幅あたりの消費電力[Wh]
    loss_elect_time.append(ship1.total_loss_time) #電力消費時間（航行時間）[hour]
    total_loss_elect.append(ship1.total_loss_elect) #総消費電力[Wh]

    ship1.storage_percentage = (ship1.storage / ship1.max_storage) * 100
    ship1.storage_state = get_storage_state(ship1.storage_percentage)
    GS_elect_storage_percentage.append(ship1.storage_percentage) #船内蓄電割合[%]
    GS_storage_state.append(ship1.storage_state)

    balance_gene_elect.append(ship1.storage) #発電収支（船内蓄電量）[Wh]

    year_round_balance_gene_elect.append(ship1.total_gene_elect - ship1.total_loss_elect) #通年発電収支


    GS_data = pl.DataFrame({"unixtime":unix,"datetime":date
                            ,"TARGET LOCATION":target_name_list,"TARGET LAT":target_lat_list,"TARGET LON":target_lon_list,"TARGET DISTANCE[km]":target_dis_list
                            ,"TARGET TYPHOON":target_typhoon_num,"TARGET TY LAT":TY_lat_list,"TARGET TY LON":TY_lon_list
                            ,"TPGSHIP LAT":GS_lat_list,"TPGSHIP LON":GS_lon_list,"TPG_TY DISTANCE[km]":GS_TY_dis_list,"BRANCH CONDITION":branch_condition_list,"TPGSHIP STATUS":GS_state_list,"SHIP SPEED[kt]":GS_speed_list
                            ,"TIMESTEP POWER GENERATION[Wh]":per_timestep_gene_elect,"TOTAL GENE TIME[h]":gene_elect_time,"TOTAL POWER GENERATION[Wh]":total_gene_elect
                            ,"TIMESTEP POWER CONSUMPTION[Wh]":per_timestep_loss_elect,"TOTAL CONS TIME[h]":loss_elect_time,"TOTAL POWER CONSUMPTION[Wh]":total_loss_elect
                            ,"ONBOARD POWER STORAGE PER[%]":GS_elect_storage_percentage,"ONBOARD POWER STORAGE STATUS":GS_storage_state,"ONBOARD ENERGY STORAGE[Wh]":balance_gene_elect,"YEARLY POWER GENERATION BALANCE":year_round_balance_gene_elect})

GS_data.write_csv(str(year) + '_sim_ver4.csv')        
                    
print("END")





END


In [ ]:
gene_out_list = []
wind_power_per_list = []
storage_list = []
sum_power_list = []
gene_list = []
for gene_out_i in range(8):
    #TPGship.generator_output = (0.14 + 0.005*gene_out_i)*(10**9) #定格出力[W]
    wind_power_per = 0.6 + 0.1 * gene_out_i
    for storage_i in range(31):
        TPGship.max_storage = (80+storage_i)*(10**9) #蓄電容量[Wh]
        five_year_gene = 0

        for year_i in range(5):
            year = 2018 + year_i
            time_step = 6
            UTC = timezone(timedelta(hours =+ 0),"UTC")
            datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
            current_time = int(datetime_1_1.timestamp())
            #終了時刻
            datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
            unixtime_12_31 = int(datetime_12_31.timestamp())
            #unixtimeでの時間幅
            time_step_unix = 3600 * time_step

            #繰り返しの回数
            record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)
            #台風データ設定
            Forecaster.forecast_time = 24*5
            Forecaster.slope = 0.0
            TY_data = Forecaster()
            TY_data.year = year
            typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
            TY_data.original_data = typhoon_data

            #発電船パラメータ設定

            max_speed_kt = 20
            #TPGship.max_storage = 21*(10**9) #蓄電容量[Wh]
            TPGship.generator_output = 0.149*(10**9) #定格出力[W]
            TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
            TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W] (今回は定格出力の1％が停止状態での抵抗)
            TPGship.wind_propulsion_power = TPGship.max_speed_power*wind_power_per #風力推進機による推進力[W]

            sum_power = TPGship.max_speed_power + TPGship.generating_facilities_need_max_power - TPGship.wind_propulsion_power
            if sum_power <= 0 :
                sum_power = 0


            ship1 = TPGship()
            ship1.forecast_time = Forecaster.forecast_time


            #拠点位置に関する設定
            #発電船拠点位置
            #沖ノ鳥島
            ship1.base_lat = 20
            ship1.base_lon = 136


            ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)
            #待機位置に関する設定
            ship1.standby_lat = 5
            ship1.standby_lon = 130

            ship1.set_initial_states()

            #外部初期値入力
            storage_state_num = get_storage_state(ship1.storage_percentage)


            for data_num in range(record_count):

                #予報データ取得
                ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                #timestep後の発電船の状態を取得
                ship1.get_next_ship_state(year , current_time , time_step)

                #timestep後の時刻の取得
                current_time = current_time + time_step_unix

            five_year_gene = five_year_gene + ship1.total_gene_elect - ship1.total_loss_elect

        
        agp = five_year_gene/5
        storage_list.append(TPGship.max_storage/10**9)
        sum_power_list.append(sum_power/10**6)
        gene_list.append(agp/10**9)
        gene_out_list.append(TPGship.generator_output/10**9)
        wind_power_per_list.append(wind_power_per*100)


        output_data = pl.DataFrame({"OUTPUT[GWh]":gene_out_list,"WINDPOWERPER[%]":wind_power_per_list,"SUMPOWER(MAXSPEED)[MWh]":sum_power_list,"STORAGE[GWh]":storage_list,"AGP[GWh]":gene_list})
        
        output_data.write_csv("test_survey_mitsuyukiSHIP_hydrostock_Pwind.csv")


In [ ]:
gene_out_list = []
wind_power_per_list = []
storage_list = []
sum_power_list = []
gene_list = []
gene_day_list = []
standby_lat_list = []
standby_lon_list = []
w_list = []
enesto_list = []
times_list = []

start_year = 2017
continue_count = 3

for storage_i in range(31):
    TPGship.max_storage = (50+storage_i)*(10**9) #蓄電容量[Wh]
    

    #台風データ設定
    Forecaster.forecast_time = 24*5
    Forecaster.slope = 0.0
    TY_data = Forecaster()
    

    #発電船パラメータ設定
    wind_power_per = 0.1
    max_speed_kt = 20
    #TPGship.max_storage = 21*(10**9) #蓄電容量[Wh]
    TPGship.generator_output = 0.138*(10**9) #定格出力[W]
    TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
    TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W] (今回は定格出力の1％が停止状態での抵抗)
    TPGship.wind_propulsion_power = TPGship.max_speed_power*wind_power_per #風力推進機による推進力[W]

    sum_power = TPGship.max_speed_power + TPGship.generating_facilities_need_max_power - TPGship.wind_propulsion_power
    if sum_power <= 0 :
        sum_power = 0


    ship1 = TPGship()
    ship1.forecast_time = Forecaster.forecast_time


    #拠点位置に関する設定
    #発電船拠点位置
    #沖ノ鳥島
    ship1.base_lat = 20
    ship1.base_lon = 136


    ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)
    for lat_i in range(3):
        #待機位置に関する設定
        ship1.standby_lat = 5 + 5*lat_i

        for lon_i in range(3):
            ship1.standby_lon = 130 + 5*lon_i

            

            for f_i in range(6):
                forecast_weight = 30 + 5*f_i

                for sto_i in range(5):
                    
                    sub_judge_energy_storage_per = 20 + 5*sto_i

                    for times_i in range(5):
                        judge_time_times = 1.0 + 0.5*times_i
                        five_year_gene = 0
                        five_year_gene_day = 0
                        for year_i in range(continue_count):
                            year = start_year + year_i
                            time_step = 6
                            UTC = timezone(timedelta(hours =+ 0),"UTC")
                            datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
                            current_time = int(datetime_1_1.timestamp())
                            #終了時刻
                            datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
                            unixtime_12_31 = int(datetime_12_31.timestamp())
                            #unixtimeでの時間幅
                            time_step_unix = 3600 * time_step

                            TY_data.year = year
                            typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
                            TY_data.original_data = typhoon_data

                            #繰り返しの回数
                            record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)

                            ship1.set_initial_states()
                            ship1.forecast_weight = forecast_weight
                            ship1.sub_judge_energy_storage_per = sub_judge_energy_storage_per
                            ship1.judge_time_times = judge_time_times
                            #外部初期値入力
                            storage_state_num = get_storage_state(ship1.storage_percentage)




                            for data_num in range(record_count):

                                #予報データ取得
                                ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                                #timestep後の発電船の状態を取得
                                ship1.get_next_ship_state(year , current_time , time_step)

                                #timestep後の時刻の取得
                                current_time = current_time + time_step_unix

                            five_year_gene = five_year_gene + ship1.total_gene_elect - ship1.total_loss_elect
                            five_year_gene_day = five_year_gene_day + ship1.total_gene_time


                        agp = five_year_gene
                        sumday = five_year_gene_day/24
                        storage_list.append(TPGship.max_storage/10**9)
                        sum_power_list.append(sum_power/10**6)
                        gene_list.append(agp/10**9)
                        gene_out_list.append(TPGship.generator_output/10**9)
                        gene_day_list.append(sumday)
                        wind_power_per_list.append(wind_power_per*100)
                        standby_lat_list.append(ship1.standby_lat)
                        standby_lon_list.append(ship1.standby_lon)
                        w_list.append(ship1.forecast_weight)
                        enesto_list.append(ship1.sub_judge_energy_storage_per)
                        times_list.append(ship1.judge_time_times)


                        output_data = pl.DataFrame({"STANDBYLAT":standby_lat_list,"STANDBYLON":standby_lon_list,"WEIGHT":w_list,"SUBSTOPER[%]":enesto_list,"TIMES":times_list,"OUTPUT[GWh]":gene_out_list,"WINDPOWERPER[%]":wind_power_per_list,"SUMPOWER(MAXSPEED)[MWh]":sum_power_list,"STORAGE[GWh]":storage_list,"SUMAGP[GWh]":gene_list,"SUMAGT[day]":gene_day_list})
                        
                        output_data.write_csv("6years_survey_case_mitsuyukiSHIP_ver3_2017-2019.csv")


In [8]:
gene_out_list = []
wind_power_per_list = []
storage_list = []
sum_power_list = []
gene_list = []
gene_day_list = []
standby_lat_list = []
standby_lon_list = []
w_list = []
enesto_list = []
times_list = []

year = 2017

for storage_i in range(10):
    TPGship.max_storage = (66+storage_i)*(10**9) #蓄電容量[Wh]
    

    #台風データ設定
    Forecaster.forecast_time = 24*5
    Forecaster.slope = 0.0
    TY_data = Forecaster()
    

    #発電船パラメータ設定
    wind_power_per = 0.1
    max_speed_kt = 20
    #TPGship.max_storage = 21*(10**9) #蓄電容量[Wh]
    TPGship.generator_output = 0.138*(10**9) #定格出力[W]
    TPGship.max_speed_power = cal_maxspeedpower(max_speed_kt,TPGship.max_storage,2,2) #船体を最大船速で進めるための出力[W]
    TPGship.generating_facilities_need_max_power = TPGship.generator_output * 0.01 #発電付加物分抵抗[W] (今回は定格出力の1％が停止状態での抵抗)
    TPGship.wind_propulsion_power = TPGship.max_speed_power*wind_power_per #風力推進機による推進力[W]

    sum_power = TPGship.max_speed_power + TPGship.generating_facilities_need_max_power - TPGship.wind_propulsion_power
    if sum_power <= 0 :
        sum_power = 0


    ship1 = TPGship()
    ship1.forecast_time = Forecaster.forecast_time


    #拠点位置に関する設定
    #発電船拠点位置
    #沖ノ鳥島
    ship1.base_lat = 20
    ship1.base_lon = 136


    
    for lat_i in range(3):
        #待機位置に関する設定
        ship1.standby_lat = 5 + 5*lat_i

        for lon_i in range(3):
            ship1.standby_lon = 130 + 5*lon_i

            

            for f_i in range(6):
                forecast_weight = 30 + 5*f_i

                for sto_i in range(5):
                    
                    sub_judge_energy_storage_per = 15 + 5*sto_i

                    for times_i in range(3):
                        judge_time_times = 1.05 + 0.05*times_i
                        five_year_gene = 0
                        five_year_gene_day = 0
                        
                        time_step = 6
                        UTC = timezone(timedelta(hours =+ 0),"UTC")
                        datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
                        current_time = int(datetime_1_1.timestamp())
                        #終了時刻
                        datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
                        unixtime_12_31 = int(datetime_12_31.timestamp())
                        #unixtimeでの時間幅
                        time_step_unix = 3600 * time_step

                        TY_data.year = year
                        typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
                        TY_data.original_data = typhoon_data
                        ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)

                        #繰り返しの回数
                        record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)

                        ship1.set_initial_states()
                        ship1.forecast_weight = forecast_weight
                        ship1.sub_judge_energy_storage_per = sub_judge_energy_storage_per
                        ship1.judge_time_times = judge_time_times
                        #外部初期値入力
                        storage_state_num = get_storage_state(ship1.storage_percentage)




                        for data_num in range(record_count):

                            #予報データ取得
                            ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

                            #timestep後の発電船の状態を取得
                            ship1.get_next_ship_state(year , current_time , time_step)

                            #timestep後の時刻の取得
                            current_time = current_time + time_step_unix

                        five_year_gene = five_year_gene + ship1.total_gene_elect - ship1.total_loss_elect
                        five_year_gene_day = five_year_gene_day + ship1.total_gene_time


                        agp = five_year_gene
                        sumday = five_year_gene_day/24
                        storage_list.append(TPGship.max_storage/10**9)
                        sum_power_list.append(sum_power/10**6)
                        gene_list.append(agp/10**9)
                        gene_out_list.append(TPGship.generator_output/10**9)
                        gene_day_list.append(sumday)
                        wind_power_per_list.append(wind_power_per*100)
                        standby_lat_list.append(ship1.standby_lat)
                        standby_lon_list.append(ship1.standby_lon)
                        w_list.append(ship1.forecast_weight)
                        enesto_list.append(ship1.sub_judge_energy_storage_per)
                        times_list.append(ship1.judge_time_times)


                        output_data = pl.DataFrame({"STANDBYLAT":standby_lat_list,"STANDBYLON":standby_lon_list,"WEIGHT":w_list,"SUBSTOPER[%]":enesto_list,"TIMES":times_list,"OUTPUT[GWh]":gene_out_list,"WINDPOWERPER[%]":wind_power_per_list,"SUMPOWER(MAXSPEED)[MWh]":sum_power_list,"STORAGE[GWh]":storage_list,"SUMAGP[GWh]":gene_list,"SUMAGT[day]":gene_day_list})
                    
                        output_data.write_csv(str(year) +"_survey_case_mitsuyukiSHIP_ver4.csv")


Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance./var/folders/8z/gkrqdthx0jg8nv7n9x_l5kvr0000gn/T/ipykernel_82388/2429183663.py:252: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  forecast_data = pl.DataFrame((zip(unix_list,ty_num_list,lat_list,lon_list,forecast_lat_list,forecast_lon_list))
/var/folders/8z/gkrqdthx0jg8nv7n9x_l5kvr0000gn/T/ipykernel_82388/1519667855.py:504: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  typhoon_data_forecast = typhoon_data_forecast.with_column(pl.Series(projected_elect_gene_time).alias("FORE_GENE_TIME"))
/var/folders/8z/gkrqdthx0jg8nv7n9x_l5kvr0000gn/T/ipykernel_82388/1519667855.py:541: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  typhoon_data_forecast = typhoon

不可侵領域設定

In [3]:
time_step = 6
year = 2022

typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")

cheat_data = typhoon_data.filter((((pl.col("LAT") >= 0) & (pl.col("LAT") <= 13)) & (pl.col("LON") >= 127.5)) | # p1 ~ p2
                                                (((pl.col("LAT") >= 13) & (pl.col("LAT") <= 15)) & (pl.col("LON") >= 125)) | # p25 ~ p255
                                                (((pl.col("LAT") >= 15) & (pl.col("LAT") <= 24)) & (pl.col("LON") >= 123)) | # p3 ~ p4
                                                (((pl.col("LAT") >= 24) & (pl.col("LAT") <= 26)) & (pl.col("LON") >= 126)) | # p5 ~ p55
                                                (((pl.col("LAT") >= 26) & (pl.col("LAT") <= 28)) & (pl.col("LON") >= 130.1)) | # p555 ~ p6
                                                (((pl.col("LAT") >= 28) & (pl.col("LAT") <= 32.2)) & (pl.col("LON") >= 132.4)) | # p7 ~ p8
                                                (((pl.col("LAT") >= 32.2) & (pl.col("LAT") <= 34)) & (pl.col("LON") >= 137.2)) | # p9 ~ p10
                                                (((pl.col("LAT") >= 34) & (pl.col("LAT") <= 41.2)) & (pl.col("LON") >= 143)) | # p11 ~ p12
                                                (((pl.col("LAT") >= 41.2) & (pl.col("LAT") <= 44)) & (pl.col("LON") >= 149)) | # p13 ~ p14
                                                (((pl.col("LAT") >= 44) & (pl.col("LAT") <= 50)) & (pl.col("LON") >= 156)) | # p15 ~ p16
                                                ((pl.col("LAT") >= 50)) # p16 ~
                                                )

cheat_data.write_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/cheat" + str(year) + "_" + str(int(time_step)) + "_interval.csv") 

Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.

台風番号振り直し

In [ ]:
time_step = 6
year = 2021

typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/cheat" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")

rep_TYnum = (year - 2000)*100 + 1

#手動でやってしまったので今後実装


In [22]:
#共通の引数設定
year = 2021
time_step = 6
UTC = timezone(timedelta(hours =+ 0),"UTC")
datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
current_time = int(datetime_1_1.timestamp())
#終了時刻
datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
unixtime_12_31 = int(datetime_12_31.timestamp())

#unixtimeでの時間幅
time_step_unix = 3600 * time_step

#繰り返しの回数
record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)

#台風データ設定
Forecaster.forecast_time = 24*5
Forecaster.slope = 0
TY_data = Forecaster()
TY_data.year = year
typhoon_data = pl.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals_verpl/cheat" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")

TY_data.original_data = typhoon_data

#発電船パラメータ設定
ship_body_num = 2
dwt = 3500
propeller_radius = 5
generation_efficiency = 0.3
hydro_generation_efficiency = 0.4

TPGship.max_storage = cal_max_storage(ship_body_num,dwt)
TPGship.power_consumption_coefficient = cal_power_consumption_coefficient(dwt,ship_body_num,hydro_generation_efficiency)
TPGship.power_generation_coefficient = cal_power_generation_coefficient(ship_body_num,propeller_radius,generation_efficiency)

ship1 = TPGship()
ship1.forecast_time = Forecaster.forecast_time
ship1.forecast_weight = 45
ship1.distance_judge_hours = 72
ship1.effective_range = 100
ship1.judge_energy_storage_per = 75


#拠点位置に関する設定
#発電船拠点位置
ship1.base_lat = 15
ship1.base_lon = 130


ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)
#待機位置に関する設定
ship1.standby_lat = 5
ship1.standby_lon = 130

ship1.set_initial_states()

#外部初期値入力
storage_state_num = get_storage_state(ship1.storage_percentage)


#####################################  出力用の設定  ############################################
#発電船の行動詳細
branch_condition_list = []

#台風の番号
target_typhoon_num = [] #そのときに追従している台風の番号（ない場合は0が入る）

#目標地点
target_name_list = []
target_lat_list = []
target_lon_list = []
target_dis_list = []

#台風座標
TY_lat_list = []
TY_lon_list = []

#発電船台風間距離
GS_TY_dis_list = []

#発電船の座標
GS_lat_list = []
GS_lon_list = []

#時刻関係
unix = [] #unixtime
date = [] #datetime

#発電船の状態
GS_state_list = [] #発電船の行動状態(描画用数値)
GS_speed_list = []

############################# 発電指数 ###############################
GS_elect_storage_percentage = [] #船内蓄電割合
GS_storage_state = []
gene_elect_time = [] #発電時間
total_gene_elect = [] #総発電量
loss_elect_time = [] #電力消費時間（航行時間）
total_loss_elect = [] #総消費電力
balance_gene_elect = [] #発電収支（船内蓄電量）
per_timestep_gene_elect = [] #時間幅あたりの発電量
per_timestep_loss_elect = [] #時間幅あたりの消費電力
year_round_balance_gene_elect  = [] #通年発電収支


#######################################  出力用リストへ入力  ###########################################

branch_condition_list.append(ship1.brance_condition)
unix.append(current_time)
date.append(datetime.fromtimestamp(unix[-1],UTC))

target_name_list.append(ship1.target_name)
target_lat_list.append(ship1.target_lat)
target_lon_list.append(ship1.target_lon)
target_dis_list.append(ship1.target_distance)

target_typhoon_num.append(ship1.target_TY)
TY_lat_list.append(ship1.next_TY_lat)
TY_lon_list.append(ship1.next_TY_lon)
GS_TY_dis_list.append(ship1.next_ship_TY_dis)

GS_lat_list.append(ship1.ship_lat)
GS_lon_list.append(ship1.ship_lon)
GS_state_list.append(ship1.ship_state)
GS_speed_list.append(ship1.speed_kt)

per_timestep_gene_elect.append(ship1.gene_elect) #時間幅あたりの発電量[Wh]
gene_elect_time.append(ship1.total_gene_time) #発電時間[hour]
total_gene_elect.append(ship1.total_gene_elect) #総発電量[Wh]

per_timestep_loss_elect.append(ship1.loss_elect) #時間幅あたりの消費電力[Wh]
loss_elect_time.append(ship1.total_loss_time) #電力消費時間（航行時間）[hour]
total_loss_elect.append(ship1.total_loss_elect) #総消費電力[Wh]

ship1.storage_percentage = (ship1.storage / ship1.max_storage) * 100
ship1.storage_state = get_storage_state(ship1.storage_percentage)
GS_elect_storage_percentage.append(ship1.storage_percentage) #船内蓄電割合[%]
GS_storage_state.append(ship1.storage_state)

balance_gene_elect.append(ship1.storage) #発電収支（船内蓄電量）[Wh]

year_round_balance_gene_elect.append(ship1.total_gene_elect - ship1.total_loss_elect) #通年発電収支
  

GS_data = pl.DataFrame({"unixtime":unix,"datetime":date
                            ,"TARGET LOCATION":target_name_list,"TARGET LAT":target_lat_list,"TARGET LON":target_lon_list,"TARGET DISTANCE[km]":target_dis_list
                            ,"TARGET TYPHOON":target_typhoon_num,"TARGET TY LAT":TY_lat_list,"TARGET TY LON":TY_lon_list
                            ,"TPGSHIP LAT":GS_lat_list,"TPGSHIP LON":GS_lon_list,"TPG_TY DISTANCE[km]":GS_TY_dis_list,"BRANCH CONDITION":branch_condition_list,"TPGSHIP STATUS":GS_state_list,"SHIP SPEED[kt]":GS_speed_list
                            ,"TIMESTEP POWER GENERATION[Wh]":per_timestep_gene_elect,"TOTAL GENE TIME[h]":gene_elect_time,"TOTAL POWER GENERATION[Wh]":total_gene_elect
                            ,"TIMESTEP POWER CONSUMPTION[Wh]":per_timestep_loss_elect,"TOTAL CONS TIME[h]":loss_elect_time,"TOTAL POWER CONSUMPTION[Wh]":total_loss_elect
                            ,"ONBOARD POWER STORAGE PER[%]":GS_elect_storage_percentage,"ONBOARD POWER STORAGE STATUS":GS_storage_state,"ONBOARD ENERGY STORAGE[Wh]":balance_gene_elect,"YEARLY POWER GENERATION BALANCE":year_round_balance_gene_elect})


for data_num in range(record_count):

    #予報データ取得
    ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

    #timestep後の発電船の状態を取得
    ship1.get_next_ship_state(year , current_time , time_step)

    #timestep後の時刻の取得
    current_time = current_time + time_step_unix

    #######################################  出力用リストへ入力  ###########################################

    branch_condition_list.append(ship1.brance_condition)
    unix.append(current_time)
    date.append(datetime.fromtimestamp(unix[-1],UTC))

    target_name_list.append(ship1.target_name)
    target_lat_list.append(ship1.target_lat)
    target_lon_list.append(ship1.target_lon)
    target_dis_list.append(ship1.target_distance)

    target_typhoon_num.append(ship1.target_TY)
    TY_lat_list.append(ship1.next_TY_lat)
    TY_lon_list.append(ship1.next_TY_lon)
    GS_TY_dis_list.append(ship1.next_ship_TY_dis)

    GS_lat_list.append(ship1.ship_lat)
    GS_lon_list.append(ship1.ship_lon)
    GS_state_list.append(ship1.ship_state)
    GS_speed_list.append(ship1.speed_kt)

    per_timestep_gene_elect.append(ship1.gene_elect) #時間幅あたりの発電量[Wh]
    gene_elect_time.append(ship1.total_gene_time) #発電時間[hour]
    total_gene_elect.append(ship1.total_gene_elect) #総発電量[Wh]

    per_timestep_loss_elect.append(ship1.loss_elect) #時間幅あたりの消費電力[Wh]
    loss_elect_time.append(ship1.total_loss_time) #電力消費時間（航行時間）[hour]
    total_loss_elect.append(ship1.total_loss_elect) #総消費電力[Wh]

    ship1.storage_percentage = (ship1.storage / ship1.max_storage) * 100
    ship1.storage_state = get_storage_state(ship1.storage_percentage)
    GS_elect_storage_percentage.append(ship1.storage_percentage) #船内蓄電割合[%]
    GS_storage_state.append(ship1.storage_state)

    balance_gene_elect.append(ship1.storage) #発電収支（船内蓄電量）[Wh]

    year_round_balance_gene_elect.append(ship1.total_gene_elect - ship1.total_loss_elect) #通年発電収支


    GS_data = pl.DataFrame({"unixtime":unix,"datetime":date
                            ,"TARGET LOCATION":target_name_list,"TARGET LAT":target_lat_list,"TARGET LON":target_lon_list,"TARGET DISTANCE[km]":target_dis_list
                            ,"TARGET TYPHOON":target_typhoon_num,"TARGET TY LAT":TY_lat_list,"TARGET TY LON":TY_lon_list
                            ,"TPGSHIP LAT":GS_lat_list,"TPGSHIP LON":GS_lon_list,"TPG_TY DISTANCE[km]":GS_TY_dis_list,"BRANCH CONDITION":branch_condition_list,"TPGSHIP STATUS":GS_state_list,"SHIP SPEED[kt]":GS_speed_list
                            ,"TIMESTEP POWER GENERATION[Wh]":per_timestep_gene_elect,"TOTAL GENE TIME[h]":gene_elect_time,"TOTAL POWER GENERATION[Wh]":total_gene_elect
                            ,"TIMESTEP POWER CONSUMPTION[Wh]":per_timestep_loss_elect,"TOTAL CONS TIME[h]":loss_elect_time,"TOTAL POWER CONSUMPTION[Wh]":total_loss_elect
                            ,"ONBOARD POWER STORAGE PER[%]":GS_elect_storage_percentage,"ONBOARD POWER STORAGE STATUS":GS_storage_state,"ONBOARD ENERGY STORAGE[Wh]":balance_gene_elect,"YEARLY POWER GENERATION BALANCE":year_round_balance_gene_elect})


GS_data.write_csv("test_" + str(year) + '_' + str(dwt) + 't_' + str(propeller_radius) + 'm_' + 'constant_speed_' + str(Forecaster.forecast_time) + 'h_不可侵領域あり.csv')        
                    
print("END")




Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance./var/folders/8z/gkrqdthx0jg8nv7n9x_l5kvr0000gn/T/ipykernel_8511/2258034941.py:146: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  GS_data = pl.DataFrame((zip(unix,date
/var/folders/8z/gkrqdthx0jg8nv7n9x_l5kvr0000gn/T/ipykernel_8511/2429183663.py:252: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  forecast_data = pl.DataFrame((zip(unix_list,ty_num_list,lat_list,lon_list,forecast_lat_list,forecast_lon_list))
/var/folders/8z/gkrqdthx0jg8nv7n9x_l5kvr0000gn/T/ipykernel_8511/2258034941.py:211: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  GS_data = pl.DataFrame((zip(unix,date
/var/folders/8z/gkrqdthx0jg8nv7n9x_l5kvr0000gn/T/ipykernel_8511/2357955248.py:424: DeprecationWarning: `with_column` has been deprecated in fa

END


2隻に増やした場合のもの

In [18]:
#共通の引数設定
year = 2019
time_step = 6
UTC = timezone(timedelta(hours =+ 0),"UTC")
datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
current_time = int(datetime_1_1.timestamp())
#終了時刻
datetime_12_31 = datetime(year,12,31,18,0,0,tzinfo = tz.gettz("UTC"))
unixtime_12_31 = int(datetime_12_31.timestamp())

#unixtimeでの時間幅
time_step_unix = 3600 * time_step

#繰り返しの回数
record_count = int( (unixtime_12_31 - current_time) / (time_step_unix) + 1)

#台風データ設定
TY_data = Forecaster()
TY_data.year = year
typhoon_data = pd.read_csv("typhoon_data_" + str(int(time_step)) + "hour_intervals/table" + str(year) + "_" + str(int(time_step)) + "_interval.csv",encoding="shift-jis")
add_unixtime(typhoon_data)
TY_data.original_data = typhoon_data

##################  1隻目  #####################
#発電船パラメータ設定
ship1 = TPGship()
ship1.forecast_time = Forecaster.forecast_time
#発電船状態量設定
ship1.set_initial_states()

#待機位置に関する設定
ship1.standby_lat = 14.2
ship1.standby_lon = 137.48 - 7.48

ship1.TY_start_time_list = get_TY_start_time(year,typhoon_data)


##################  2隻目  #####################
#発電船パラメータ設定
ship2 = TPGship()
ship2.forecast_time = Forecaster.forecast_time
#発電船状態量設定
ship2.set_initial_states()

#待機位置に関する設定
ship2.standby_lat = 14.2 + 5.8
ship2.standby_lon = 137.48

ship2.TY_start_time_list = get_TY_start_time(year,typhoon_data)


#####################################  出力用の設定  ############################################

###################  1隻目  #######################
#発電船の行動詳細
branch_condition_list = []

#台風の番号
target_typhoon_num = [] #そのときに追従している台風の番号（ない場合は0が入る）

#目標地点
target_name_list = []
target_lat_list = []
target_lon_list = []
target_dis_list = []

#台風座標
TY_lat_list = []
TY_lon_list = []

#発電船台風間距離
GS_TY_dis_list = []

#発電船の座標
GS_lat_list = []
GS_lon_list = []

#時刻関係
unix = [] #unixtime
date = [] #datetime

#発電船の状態
GS_state_list = [] #発電船の行動状態(描画用数値)
GS_speed_list = []

############################# 発電指数 ###############################
GS_elect_storage_percentage = [] #船内蓄電割合
GS_storage_state = []
gene_elect_time = [] #発電時間
total_gene_elect = [] #総発電量
loss_elect_time = [] #電力消費時間（航行時間）
total_loss_elect = [] #総消費電力
balance_gene_elect = [] #発電収支（船内蓄電量）
per_timestep_gene_elect = [] #時間幅あたりの発電量
per_timestep_loss_elect = [] #時間幅あたりの消費電力
year_round_balance_gene_elect  = [] #通年発電収支




###################  2隻目  #######################
#発電船の行動詳細
branch_condition_list2 = []

#台風の番号
target_typhoon_num2 = [] #そのときに追従している台風の番号（ない場合は0が入る）

#目標地点
target_name_list2 = []
target_lat_list2 = []
target_lon_list2 = []
target_dis_list2 = []

#台風座標
TY_lat_list2 = []
TY_lon_list2 = []

#発電船台風間距離
GS_TY_dis_list2 = []

#発電船の座標
GS_lat_list2 = []
GS_lon_list2 = []


#発電船の状態
GS_state_list2 = [] #発電船の行動状態(描画用数値)
GS_speed_list2 = []

############################# 発電指数 ###############################
GS_elect_storage_percentage2 = [] #船内蓄電割合
GS_storage_state2 = []
gene_elect_time2 = [] #発電時間
total_gene_elect2 = [] #総発電量
loss_elect_time2 = [] #電力消費時間（航行時間）
total_loss_elect2 = [] #総消費電力
balance_gene_elect2 = [] #発電収支（船内蓄電量）
per_timestep_gene_elect2 = [] #時間幅あたりの発電量
per_timestep_loss_elect2 = [] #時間幅あたりの消費電力
year_round_balance_gene_elect2  = [] #通年発電収支










#######################################  出力用リストへ入力  ###########################################


####################  1隻目  #######################

branch_condition_list.append(ship1.brance_condition)
unix.append(current_time)
date.append(datetime.fromtimestamp(unix[-1],UTC))

target_name_list.append(ship1.target_name)
target_lat_list.append(ship1.target_lat)
target_lon_list.append(ship1.target_lon)
target_dis_list.append(ship1.target_distance)

target_typhoon_num.append(ship1.target_TY)
TY_lat_list.append(ship1.next_TY_lat)
TY_lon_list.append(ship1.next_TY_lon)
GS_TY_dis_list.append(ship1.next_ship_TY_dis)

GS_lat_list.append(ship1.ship_lat)
GS_lon_list.append(ship1.ship_lon)
GS_state_list.append(ship1.ship_state)
GS_speed_list.append(ship1.speed_kt)

per_timestep_gene_elect.append(ship1.gene_elect) #時間幅あたりの発電量[Wh]
gene_elect_time.append(ship1.total_gene_time) #発電時間[hour]
total_gene_elect.append(ship1.total_gene) #総発電量[Wh]

per_timestep_loss_elect.append(ship1.loss_elect) #時間幅あたりの消費電力[Wh]
loss_elect_time.append(ship1.total_loss_time) #電力消費時間（航行時間）[hour]
total_loss_elect.append(ship1.total_loss) #総消費電力[Wh]

ship1.storage_percentage = (ship1.storage / ship1.max_storage) * 100
ship1.storage_state = ship1.get_storage_state()
GS_elect_storage_percentage.append(ship1.storage_percentage) #船内蓄電割合[%]
GS_storage_state.append(ship1.storage_state)

balance_gene_elect.append(ship1.storage) #発電収支（船内蓄電量）[Wh]

year_round_balance_gene_elect.append(ship1.total_gene - ship1.total_loss) #通年発電収支


GS_data = pd.DataFrame((zip(unix,date
                            ,target_name_list,target_lat_list,target_lon_list,target_dis_list
                            ,target_typhoon_num,TY_lat_list,TY_lon_list
                            ,GS_lat_list,GS_lon_list,GS_TY_dis_list,branch_condition_list,GS_state_list,GS_speed_list
                            ,per_timestep_gene_elect,gene_elect_time,total_gene_elect
                            ,per_timestep_loss_elect,loss_elect_time,total_loss_elect
                            ,GS_elect_storage_percentage,GS_storage_state,balance_gene_elect,year_round_balance_gene_elect))
                            ,columns=["unixtime","datetime"
                            ,"目標地点名","目標地点緯度","目標地点経度","目標地点距離[km]"
                            ,"追従対象台風","対象台風緯度","対象台風経度","発電船緯度","発電船経度","発電船台風間距離[km]","分岐条件位置","発電船状態","船速[kt]"
                            ,"時間幅発電量[Wh]","発電時間[h]","総発電量[Wh]"
                            ,"時間幅消費電力[Wh]","電力消費時間[h]","総消費電力[Wh]"
                            ,"船内蓄電割合[%]","船内蓄電状態","船内蓄電量[Wh]","通年発電収支"])   



####################  2隻目  #######################

branch_condition_list2.append(ship2.brance_condition)

target_name_list2.append(ship2.target_name)
target_lat_list2.append(ship2.target_lat)
target_lon_list2.append(ship2.target_lon)
target_dis_list2.append(ship2.target_distance)

target_typhoon_num2.append(ship2.target_TY)
TY_lat_list2.append(ship2.next_TY_lat)
TY_lon_list2.append(ship2.next_TY_lon)
GS_TY_dis_list2.append(ship2.next_ship_TY_dis)

GS_lat_list2.append(ship2.ship_lat)
GS_lon_list2.append(ship2.ship_lon)
GS_state_list2.append(ship2.ship_state)
GS_speed_list2.append(ship2.speed_kt)

per_timestep_gene_elect2.append(ship2.gene_elect) #時間幅あたりの発電量[Wh]
gene_elect_time2.append(ship2.total_gene_time) #発電時間[hour]
total_gene_elect2.append(ship2.total_gene) #総発電量[Wh]

per_timestep_loss_elect2.append(ship2.loss_elect) #時間幅あたりの消費電力[Wh]
loss_elect_time2.append(ship2.total_loss_time) #電力消費時間（航行時間）[hour]
total_loss_elect2.append(ship2.total_loss) #総消費電力[Wh]

ship2.storage_percentage = (ship2.storage / ship2.max_storage) * 100
ship2.storage_state = ship2.get_storage_state()
GS_elect_storage_percentage2.append(ship2.storage_percentage) #船内蓄電割合[%]
GS_storage_state2.append(ship2.storage_state)

balance_gene_elect2.append(ship2.storage) #発電収支（船内蓄電量）[Wh]

year_round_balance_gene_elect2.append(ship2.total_gene - ship2.total_loss) #通年発電収支


GS_data2 = pd.DataFrame((zip(unix,date
                            ,target_name_list2,target_lat_list2,target_lon_list2,target_dis_list2
                            ,target_typhoon_num2,TY_lat_list2,TY_lon_list2
                            ,GS_lat_list2,GS_lon_list2,GS_TY_dis_list2,branch_condition_list2,GS_state_list2,GS_speed_list2
                            ,per_timestep_gene_elect2,gene_elect_time2,total_gene_elect2
                            ,per_timestep_loss_elect2,loss_elect_time2,total_loss_elect2
                            ,GS_elect_storage_percentage2,GS_storage_state2,balance_gene_elect2,year_round_balance_gene_elect2))
                            ,columns=["unixtime","datetime"
                            ,"目標地点名","目標地点緯度","目標地点経度","目標地点距離[km]"
                            ,"追従対象台風","対象台風緯度","対象台風経度","発電船緯度","発電船経度","発電船台風間距離[km]","分岐条件位置","発電船状態","船速[kt]"
                            ,"時間幅発電量[Wh]","発電時間[h]","総発電量[Wh]"
                            ,"時間幅消費電力[Wh]","電力消費時間[h]","総消費電力[Wh]"
                            ,"船内蓄電割合[%]","船内蓄電状態","船内蓄電量[Wh]","通年発電収支"])   





for data_num in range(record_count):

    #予報データ取得
    ship1.forecast_data = TY_data.create_forecast(time_step,current_time)

    #timestep後の発電船の状態を取得
    ship1.get_next_ship_state(year , current_time , time_step)

    #予報データ取得（1隻目と同じもの選択可）
    #ship2.forecast_data = TY_data.create_forecast(current_time)

    #予報データ取得（1隻目が選んだものは選択不可）
    ship2.forecast_data = forecast_data_revision(ship1.target_TY,ship1.forecast_data)

    #timestep後の発電船の状態を取得
    ship2.get_next_ship_state(year , current_time , time_step)


    #timestep後の時刻の取得
    current_time = current_time + time_step_unix

    #######################################  出力用リストへ入力  ###########################################

    branch_condition_list.append(ship1.brance_condition)
    unix.append(current_time)
    date.append(datetime.fromtimestamp(unix[-1],UTC))

    target_name_list.append(ship1.target_name)
    target_lat_list.append(ship1.target_lat)
    target_lon_list.append(ship1.target_lon)
    target_dis_list.append(ship1.target_distance)

    target_typhoon_num.append(ship1.target_TY)
    TY_lat_list.append(ship1.next_TY_lat)
    TY_lon_list.append(ship1.next_TY_lon)
    GS_TY_dis_list.append(ship1.next_ship_TY_dis)

    GS_lat_list.append(ship1.ship_lat)
    GS_lon_list.append(ship1.ship_lon)
    GS_state_list.append(ship1.ship_state)
    GS_speed_list.append(ship1.speed_kt)

    per_timestep_gene_elect.append(ship1.gene_elect) #時間幅あたりの発電量[Wh]
    gene_elect_time.append(ship1.total_gene_time) #発電時間[hour]
    total_gene_elect.append(ship1.total_gene) #総発電量[Wh]

    per_timestep_loss_elect.append(ship1.loss_elect) #時間幅あたりの消費電力[Wh]
    loss_elect_time.append(ship1.total_loss_time) #電力消費時間（航行時間）[hour]
    total_loss_elect.append(ship1.total_loss) #総消費電力[Wh]

    ship1.storage_percentage = (ship1.storage / ship1.max_storage) * 100
    ship1.storage_state = ship1.get_storage_state()
    GS_elect_storage_percentage.append(ship1.storage_percentage) #船内蓄電割合[%]
    GS_storage_state.append(ship1.storage_state)

    balance_gene_elect.append(ship1.storage) #発電収支（船内蓄電量）[Wh]

    year_round_balance_gene_elect.append(ship1.total_gene - ship1.total_loss) #通年発電収支


    GS_data = pd.DataFrame((zip(unix,date
                                ,target_name_list,target_lat_list,target_lon_list,target_dis_list
                                ,target_typhoon_num,TY_lat_list,TY_lon_list
                                ,GS_lat_list,GS_lon_list,GS_TY_dis_list,branch_condition_list,GS_state_list,GS_speed_list
                                ,per_timestep_gene_elect,gene_elect_time,total_gene_elect
                                ,per_timestep_loss_elect,loss_elect_time,total_loss_elect
                                ,GS_elect_storage_percentage,GS_storage_state,balance_gene_elect,year_round_balance_gene_elect))
                                ,columns=["unixtime","datetime"
                                ,"目標地点名","目標地点緯度","目標地点経度","目標地点距離[km]"
                                ,"追従対象台風","対象台風緯度","対象台風経度","発電船緯度","発電船経度","発電船台風間距離[km]","分岐条件位置","発電船状態","船速[kt]"
                                ,"時間幅発電量[Wh]","発電時間[h]","総発電量[Wh]"
                                ,"時間幅消費電力[Wh]","電力消費時間[h]","総消費電力[Wh]"
                                ,"船内蓄電割合[%]","船内蓄電状態","船内蓄電量[Wh]","通年発電収支"])   
    


    ####################  2隻目  #######################

    branch_condition_list2.append(ship2.brance_condition)

    target_name_list2.append(ship2.target_name)
    target_lat_list2.append(ship2.target_lat)
    target_lon_list2.append(ship2.target_lon)
    target_dis_list2.append(ship2.target_distance)

    target_typhoon_num2.append(ship2.target_TY)
    TY_lat_list2.append(ship2.next_TY_lat)
    TY_lon_list2.append(ship2.next_TY_lon)
    GS_TY_dis_list2.append(ship2.next_ship_TY_dis)

    GS_lat_list2.append(ship2.ship_lat)
    GS_lon_list2.append(ship2.ship_lon)
    GS_state_list2.append(ship2.ship_state)
    GS_speed_list2.append(ship2.speed_kt)

    per_timestep_gene_elect2.append(ship2.gene_elect) #時間幅あたりの発電量[Wh]
    gene_elect_time2.append(ship2.total_gene_time) #発電時間[hour]
    total_gene_elect2.append(ship2.total_gene) #総発電量[Wh]

    per_timestep_loss_elect2.append(ship2.loss_elect) #時間幅あたりの消費電力[Wh]
    loss_elect_time2.append(ship2.total_loss_time) #電力消費時間（航行時間）[hour]
    total_loss_elect2.append(ship2.total_loss) #総消費電力[Wh]

    ship2.storage_percentage = (ship2.storage / ship2.max_storage) * 100
    ship2.storage_state = ship2.get_storage_state()
    GS_elect_storage_percentage2.append(ship2.storage_percentage) #船内蓄電割合[%]
    GS_storage_state2.append(ship2.storage_state)

    balance_gene_elect2.append(ship2.storage) #発電収支（船内蓄電量）[Wh]

    year_round_balance_gene_elect2.append(ship2.total_gene - ship2.total_loss) #通年発電収支


    GS_data2 = pd.DataFrame((zip(unix,date
                                ,target_name_list2,target_lat_list2,target_lon_list2,target_dis_list2
                                ,target_typhoon_num2,TY_lat_list2,TY_lon_list2
                                ,GS_lat_list2,GS_lon_list2,GS_TY_dis_list2,branch_condition_list2,GS_state_list2,GS_speed_list2
                                ,per_timestep_gene_elect2,gene_elect_time2,total_gene_elect2
                                ,per_timestep_loss_elect2,loss_elect_time2,total_loss_elect2
                                ,GS_elect_storage_percentage2,GS_storage_state2,balance_gene_elect2,year_round_balance_gene_elect2))
                                ,columns=["unixtime","datetime"
                                ,"目標地点名","目標地点緯度","目標地点経度","目標地点距離[km]"
                                ,"追従対象台風","対象台風緯度","対象台風経度","発電船緯度","発電船経度","発電船台風間距離[km]","分岐条件位置","発電船状態","船速[kt]"
                                ,"時間幅発電量[Wh]","発電時間[h]","総発電量[Wh]"
                                ,"時間幅消費電力[Wh]","電力消費時間[h]","総消費電力[Wh]"
                                ,"船内蓄電割合[%]","船内蓄電状態","船内蓄電量[Wh]","通年発電収支"])   

GS_data.to_csv("ship1_" + str(year) + '_' + str(ship1.dwt) + 't_' + str(ship1.propeller) + 'm_' + 'constant_speed_' + str(forecast_time) + 'h.csv',encoding="shift-jis")        

GS_data2.to_csv("ship2_" + str(year) + '_' + str(ship1.dwt) + 't_' + str(ship1.propeller) + 'm_' + 'constant_speed_' + str(forecast_time) + 'h.csv',encoding="shift-jis")        
                    
print("END")

END
